# PTM Information
As we have just considered the 'Modified Residue' when retrieving the PTM info in 'retrieve_variants_info_GnomAD_by_classes.ipynb' and we need to add the other types of PTMs (lipidation, glycosylation, etc) we create this new file. This will build a new PTM_IMPACT/PTMs_all.csv and the data will be then added to the 'final' file with all the variant info.

In [1]:
path_to_project="/home/martalo/Documentos/TFM/GPCR_variants"

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install scipy
!{sys.executable} -m pip install bioservices
!{sys.executable} -m pip install biopython
!{sys.executable} -m pip install xlrd

from common_functions import *

import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist, squareform
import urllib

import mdtraj as md
import itertools
config(viewer='webgl')
from create_csv import * # this file is from Code/create_csv.py
import requests
import os
from matplotlib.ticker import FormatStrFormatter

/home/martalo/miniconda3/envs/my_new_env_name/lib/python3.6/site-packages/htmd/versionwarnings.py:29: UserWarning: As of HTMD 1.16 the default ACEMD version for all protocols has changed to version 3. If you want to use version 2 protocols change the _version argument in the protocols or add `config(acemdversion=2)` to the beginning of your scripts. To disable this warning run once: `from htmd import _disableWarnings; _disableWarnings('1.16');`
  , UserWarning)
/home/martalo/miniconda3/envs/my_new_env_name/lib/python3.6/site-packages/htmd/versionwarnings.py:33: UserWarning: As of HTMD 1.16 the default number of threads HTMD spawns for calculations is set to 1. You can enable parallelism at your own risk using `config(njobs=-2)` in the beginning of your scripts. To disable this warning run once: `from htmd import _disableWarnings; _disableWarnings('1.16');`
  , UserWarning)


ffevaluate module is in beta version

Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049

HTMD Documentation at: https://www.htmd.org/docs/latest/

New devel HTMD version (1.24.1 python[3.6,<3.7.0a0,3.7,<3.8.0a0]) is available. You are currently on (1.19).There are several methods to update:    - Create a new conda env. using `conda create -n htmd1.24.1 htmd=1.24.1 -c acellera -c psi4 -c conda-forge`    - Create a brand new conda installation and run `conda install htmd -c acellera -c psi4 -c conda-forge`    - Run: `conda update htmd -c acellera -c psi4 -c conda-forge` (NOT RECOMMENDED)



In [2]:
def get_tranformation_features(transfName):
    '''Returns a list with the values need to query this PTM in Uniprot'''
    # Build dict with the PTM types and the corresponding info needed to query them
    # feature for the URL, split response, first line response
    PTMnames={'modRes':['MODIFIED%20RESIDUE','MOD_RES ','Modified residue'],'glyco':['GLYCOSYLATION','CARBOHYD ','Glycosylation'],
             'lipid':['LIPIDATION', 'LIPID ','Lipidation'], 'disBond':['DISULFIDE%20BOND','DISULFID ','Disulfide bond']}
    return PTMnames[transfName]


def res_groups(response, splitBy, fristLine):
    '''Build a list of list for each PTM, storing position'''
    residues=[]
    data=[]

    lines = response.split(splitBy)  
    if lines[0].startswith(fristLine):
        lines.remove(lines[0])
#     print('lines:',lines)
        if splitBy!='DISULFID ':#modres, lipid, glyco
            for element in lines:
                data=element.split(';')
                note=data[1][9:]
                if note[-1]=='\"':
                    note=note[:-1]
                residues.append([data[0],note])      
            print('res:',residues)
        else: # disulfide b
            print(lines)
            for element in lines:
                data=element.split(';')
#                 print(data)
                positions=data[0]
#                 print(positions)
                residues.append([positions,'Disulfide Bond']) 
    return(residues) # if no DB [position, description of the PTM ])
                     # if DB [ positions, 'Disulfide Bond']

def get_ptm(entry, transformation):
    '''Query Uniprot with the entry number to get the PTM information, return the PTM type if
    position correspond to one PTM, otherwise return None'''
    featureList=get_tranformation_features(transformation)# feature to URL, split response, 1st line response
    feature= featureList[0]
    splitLine=featureList[1]
    stLine=featureList[2]
    # the query needs to be specified by id, otherwise, returns all entries that somwhere contain the 
    # term, for example, Q8TDT2, e.g. proteins that interact with Q8TDT2
    # (they also recommend to show the results by entry: ...columns=id,feature....
    # so the refence entry is displayed in the first column, but I think we don't need it)
    with urllib.request.urlopen('http://www.uniprot.org/uniprot/?query=id:'+entry+'&format=tab&columns=feature('+feature+')') as response:
        mod_res = response.read().decode()
        print('Reponse from uniprot:',mod_res)
        resInfo=res_groups(mod_res,splitLine,stLine)# list of list with all the [PTM position,PTM type] for the entry
#         print(resInfo)
    return resInfo

# Example 
# get_ptm("P35348", 'modRes')

# Example 
# get_ptm("Q5NUL3")
# Returns the correct data from Uniprot 'Modified Residues':
# [['347', 'Phosphothreonine'], ['349', 'Phosphothreonine'], 
# ['350', 'Phosphoserine'], ['357', 'Phosphoserine'], ['360', 'Phosphoserine']]

def dict_ptm(var_file, transf):
    '''Build a dict with the keys as the entry names that are present in the file,
    values are lists [position, ptmtype, eco] of the corresponding PTM type we want to query '''
    ptmDic={}
    prots_without_pdb=['P30988-2', 'P34998-2']
    with open(var_file,"r") as myvarfile:
        myvarread = csv.reader(myvarfile, delimiter=';') 
        for myrow in myvarread:
            if myrow[0]=="Family": #first row, new names
                continue            
            elif myrow[3] not in prots_without_pdb: # avoid the names of non-wanted proteins
                print(myrow[3])
                ptmDic[myrow[3]]=get_ptm(myrow[3],transf)
    print(ptmDic)
    return ptmDic

In [3]:
# this is a test function to see the output from Uniprot Mod Residue just with the entry name
def get_ptm_test(entry):
    '''Query Uniprot with the entry number to get the PTM Mod Res information'''
#     featureList=get_tranformation_features(transformation)# feature to URL, split response, 1st line response
#     feature= featureList[0]
#     splitLine=featureList[1]
#     stLine=featureList[2]
    with urllib.request.urlopen('http://www.uniprot.org/uniprot/?query=id:'+entry+'&format=tab&columns=feature(MODIFIED%20RESIDUE)') as response:
        mod_res = response.read().decode()
#         print('Reponse from uniprot:',mod_res)
#         resInfo=res_groups(mod_res,splitLine,stLine)# list of list with all the [PTM position,PTM type] for the entry
        print(mod_res)
    return mod_res

# Example (old)   
# get_ptm_test("P35348")

## Modified Residue
Modified residues excluding lipids, glycans and protein crosslinks.

In [4]:
# Build the PTM modiefied residues dictionary keys as the entry names, values are lists [position, ptmtype]
dicPTMmodRes=dict_ptm(os.path.join(resultspath,'remaining3/myprot_list_struc_remaining_merged_ENSG.csv'),'modRes')

P08100
Reponse from uniprot: Modified residue
MOD_RES 1;  /note="N-acetylmethionine";  /evidence="ECO:0000250|UniProtKB:P02699"; MOD_RES 296;  /note="N6-(retinylidene)lysine";  /evidence="ECO:0000250|UniProtKB:P02699"; MOD_RES 334;  /note="Phosphoserine";  /evidence="ECO:0000269|PubMed:28753425"; MOD_RES 336;  /note="Phosphothreonine";  /evidence="ECO:0000269|PubMed:28753425"; MOD_RES 338;  /note="Phosphoserine";  /evidence="ECO:0000269|PubMed:28753425"; MOD_RES 340;  /note="Phosphothreonine";  /evidence="ECO:0000250|UniProtKB:P02699"; MOD_RES 342;  /note="Phosphothreonine";  /evidence="ECO:0000250|UniProtKB:P02699"; MOD_RES 343;  /note="Phosphoserine";  /evidence="ECO:0000250|UniProtKB:P02699"

res: [['1', 'N-acetylmethionine'], ['296', 'N6-(retinylidene)lysine'], ['334', 'Phosphoserine'], ['336', 'Phosphothreonine'], ['338', 'Phosphoserine'], ['340', 'Phosphothreonine'], ['342', 'Phosphothreonine'], ['343', 'Phosphoserine']]
Q9UHM6
Reponse from uniprot: Modified residue
MOD_RES 340; 

In [5]:
# store the dictionary to avoid downloading it again
dicPTMmodRes={'P18089': [], 'P21917': [], 'P35367': [['140', 'Phosphothreonine'], ['142', 'Phosphothreonine'], ['230', 'Phosphoserine'], ['279', 'Phosphothreonine'], ['344', 'Phosphoserine'], ['347', 'Phosphoserine'], ['380', 'Phosphoserine'], ['396', 'Phosphoserine'], ['398', 'Phosphoserine']], 'P48145': [], 'P48146': [], 'Q99463': [], 'Q9NSD7': [], 'P32745': [['332', 'Phosphoserine'], ['337', 'Phosphoserine'], ['348', 'Phosphothreonine']], 'Q15722': [], 'Q9NPC1': [], 'P0DMS8': [], 'P46091': [], 'Q99678': [], 'Q99680': [], 'Q96CH1': [], 'Q8TDV2': [], 'P35410': [], 'Q9P1P4': [], 'O00398': [], 'Q9P1P5': [], 'O14804': [], 'Q96RI8': [], 'Q969N4': [], 'Q96RI9': []}

# old: {'P08908': [], 'P28222': [], 'P28221': [], 'P28566': [], 'P30939': [], 'P28223': [['280', 'Phosphoserine']], 'P41595': [], 'P28335': [], 'Q13639': [], 'P47898': [], 'P50406': [], 'P34969': [], 'P11229': [['230', 'Phosphothreonine'], ['428', 'Phosphothreonine'], ['451', 'Phosphoserine'], ['455', 'Phosphothreonine'], ['457', 'Phosphoserine']], 'P08172': [['232', 'Phosphoserine'], ['446', 'Phosphothreonine'], ['450', 'Phosphothreonine'], ['465', 'Phosphothreonine']], 'P20309': [['385', 'Phosphoserine']], 'P08173': [['459', 'Phosphothreonine'], ['463', 'Phosphothreonine'], ['477', 'Phosphothreonine']], 'P08912': [['501', 'Phosphothreonine'], ['505', 'Phosphothreonine']], 'P35348': [['215', 'Phosphoserine']], 'P35368': [['264', 'Phosphothreonine']], 'P25100': [], 'P08913': [['346', 'Phosphoserine'], ['368', 'Omega-N-methylarginine']], 'P18089': [], 'P18825': [], 'P08588': [['312', 'Phosphoserine'], ['412', 'Phosphoserine'], ['428', 'Phosphoserine']], 'P07550': [['141', 'Phosphotyrosine'], ['246', 'Phosphoserine'], ['261', 'Phosphoserine'], ['262', 'Phosphoserine'], ['345', 'Phosphoserine'], ['346', 'Phosphoserine'], ['355', 'Phosphoserine'], ['356', 'Phosphoserine'], ['382', '4-hydroxyproline'], ['395', '4-hydroxyproline']], 'P13945': [], 'P21728': [], 'P14416': [], 'P35462': [], 'P21917': [], 'P21918': [], 'P35367': [['140', 'Phosphothreonine'], ['142', 'Phosphothreonine'], ['230', 'Phosphoserine'], ['279', 'Phosphothreonine'], ['344', 'Phosphoserine'], ['347', 'Phosphoserine'], ['380', 'Phosphoserine'], ['396', 'Phosphoserine'], ['398', 'Phosphoserine']], 'P25021': [], 'Q9Y5N1': [['439', 'Phosphoserine']], 'Q9H3N8': [], 'Q96RJ0': [], 'P30556': [], 'P50052': [], 'P35414': [], 'P28336': [['352', 'Phosphoserine']], 'P30550': [['350', 'Phosphoserine']], 'P32247': [], 'P46663': [], 'P30411': [['156', 'Phosphotyrosine'], ['347', 'Phosphotyrosine'], ['366', 'Phosphoserine'], ['369', 'Phosphothreonine'], ['373', 'Phosphoserine'], ['375', 'Phosphoserine']], 'P32238': [], 'P32239': [], 'Q16581': [['174', 'Sulfotyrosine'], ['184', 'Sulfotyrosine'], ['318', 'Sulfotyrosine'], ['459', 'Phosphoserine'], ['463', 'Phosphothreonine']], 'P21730': [['11', 'Sulfotyrosine'], ['14', 'Sulfotyrosine'], ['314', 'Phosphoserine'], ['317', 'Phosphoserine'], ['327', 'Phosphoserine'], ['332', 'Phosphoserine'], ['334', 'Phosphoserine'], ['338', 'Phosphoserine']], 'Q9P296': [['320', 'Phosphoserine']], 'P25101': [['425', 'Phosphoserine']], 'P24530': [['305', 'Phosphoserine'], ['419', 'Phosphoserine'], ['439', 'Phosphotyrosine'], ['440', 'Phosphoserine'], ['441', 'Phosphoserine'], ['442', 'Phosphoserine']], 'P21462': [['328', 'Phosphoserine'], ['329', 'Phosphothreonine'], ['331', 'Phosphothreonine'], ['332', 'Phosphoserine'], ['334', 'Phosphothreonine'], ['336', 'Phosphothreonine'], ['338', 'Phosphoserine'], ['339', 'Phosphothreonine']], 'P25090': [], 'P25089': [], 'P47211': [], 'O43603': [], 'O60755': [], 'Q92847': [], 'P30968': [], 'Q969F8': [], 'Q99705': [], 'Q969V1': [], 'Q01726': [], 'Q01718': [], 'P41968': [], 'P32245': [], 'P33032': [], 'O43193': [], 'Q9HB89': [], 'Q9GZQ4': [], 'Q9GZQ6': [], 'Q9Y5X5': [], 'Q6W5P4': [], 'P48145': [], 'P48146': [], 'P25929': [['368', 'Phosphoserine']], 'P49146': [], 'P50391': [], 'Q15761': [], 'Q99463': [], 'P30989': [], 'O95665': [], 'P41143': [], 'P41145': [], 'P35372': [['168', 'Phosphotyrosine'], ['365', 'Phosphoserine'], ['372', 'Phosphothreonine'], ['377', 'Phosphoserine'], ['396', 'Phosphothreonine']], 'P41146': [['363', 'Phosphoserine']], 'O43613': [], 'O43614': [], 'Q96P65': [], 'P49683': [], 'P25116': [['418', 'Phosphoserine']], 'P55085': [], 'O00254': [], 'Q96RI0': [], 'Q9HBX9': [], 'Q8WXD0': [], 'Q9NSD7': [], 'Q8TDU9': [], 'P30872': [], 'P30874': [['341', 'Phosphoserine'], ['343', 'Phosphoserine'], ['348', 'Phosphoserine'], ['353', 'Phosphothreonine'], ['354', 'Phosphothreonine']], 'P32745': [['332', 'Phosphoserine'], ['337', 'Phosphoserine'], ['348', 'Phosphothreonine']], 'P31391': [], 'P35346': [['325', 'Phosphoserine']], 'P25103': [], 'P21452': [], 'P29371': [], 'P34981': [], 'Q9UKP6': [], 'P37288': [['404', 'Phosphoserine']], 'P47901': [], 'P30518': [], 'P30559': [['366', 'Phosphoserine'], ['368', 'Phosphoserine']], 'Q99788': [['339', 'Phosphoserine'], ['342', 'Phosphothreonine'], ['349', 'Phosphoserine'], ['352', 'Phosphoserine'], ['358', 'Phosphoserine']], 'P32246': [], 'P41597': [['26', 'Sulfotyrosine'], ['139', 'Phosphotyrosine']], 'P51677': [], 'P51679': [], 'P51681': [['3', 'Sulfotyrosine'], ['10', 'Sulfotyrosine'], ['14', 'Sulfotyrosine'], ['15', 'Sulfotyrosine'], ['336', 'Phosphoserine'], ['337', 'Phosphoserine'], ['342', 'Phosphoserine'], ['349', 'Phosphoserine']], 'P51684': [], 'P32248': [], 'P51685': [], 'P51686': [], 'P46092': [], 'P25024': [], 'P25025': [['347', 'Phosphoserine'], ['351', 'Phosphoserine'], ['352', 'Phosphoserine'], ['353', 'Phosphoserine']], 'P49682': [['27', 'Sulfotyrosine'], ['29', 'Sulfotyrosine']], 'P61073': [['7', 'Sulfotyrosine'], ['12', 'Sulfotyrosine'], ['21', 'Sulfotyrosine'], ['319', 'Phosphoserine'], ['321', 'Phosphoserine'], ['324', 'Phosphoserine'], ['325', 'Phosphoserine'], ['330', 'Phosphoserine'], ['339', 'Phosphoserine'], ['348', 'Phosphoserine'], ['351', 'Phosphoserine']], 'P32302': [], 'O00574': [], 'P49238': [['346', 'Phosphothreonine']], 'P46094': [], 'Q16570': [], 'O00590': [], 'P25106': [['347', 'Phosphoserine'], ['350', 'Phosphoserine'], ['355', 'Phosphoserine']], 'Q9NPB9': [], 'O00421': [], 'P23945': [['335', 'Sulfotyrosine']], 'P22888': [['331', 'Sulfotyrosine']], 'P16473': [['385', 'Sulfotyrosine']], 'Q8TCW9': [], 'Q8NFJ6': [], 'O14842': [], 'O15552': [], 'O14843': [], 'Q5NUL3': [['347', 'Phosphothreonine'], ['349', 'Phosphothreonine'], ['350', 'Phosphoserine'], ['357', 'Phosphoserine'], ['360', 'Phosphoserine']], 'O15529': [], 'Q15722': [], 'Q9NPC1': [], 'Q9Y271': [], 'Q9NS75': [], 'Q8TDS5': [], 'Q92633': [['341', 'Phosphoserine'], ['351', 'Phosphothreonine']], 'Q9HBW0': [], 'Q9UBY5': [], 'Q99677': [], 'Q9H1C0': [], 'P43657': [], 'P21453': [['10', 'N6-acetyllysine'], ['236', 'Phosphothreonine'], ['351', 'Phosphoserine'], ['353', 'Phosphoserine']], 'O95136': [], 'Q99500': [['326', 'Phosphoserine']], 'O95977': [], 'Q9H228': [['381', 'Phosphoserine']], 'P21554': [['425', 'Phosphoserine'], ['429', 'Phosphoserine']], 'P34972': [['335', 'Phosphoserine'], ['336', 'Phosphoserine'], ['338', 'Phosphothreonine'], ['352', 'Phosphoserine']], 'Q14330': [['322', 'Phosphoserine']], 'Q9Y2T6': [], 'Q8TDV5': [], 'P25105': [], 'Q13258': [], 'Q9Y5Y4': [['331', 'Phosphoserine'], ['345', 'Phosphoserine']], 'P34995': [], 'P43116': [], 'P43115': [], 'P35408': [['374', 'Phosphoserine'], ['377', 'Phosphoserine'], ['379', 'Phosphoserine'], ['382', 'Phosphoserine']], 'P43088': [], 'P43119': [['383', 'Cysteine methyl ester']], 'P21731': [['329', 'Phosphoserine'], ['331', 'Phosphoserine']], 'P48039': [], 'P49286': [], 'P30542': [], 'P29274': [], 'P29275': [], 'P0DMS8': [], 'P47900': [], 'P41231': [], 'P51582': [['333', 'Phosphoserine'], ['334', 'Phosphoserine']], 'Q15077': [], 'Q96G91': [], 'Q9H244': [['55', 'Phosphoserine'], ['57', 'Phosphoserine']], 'Q9BPV8': [], 'Q15391': [], 'Q8TDU6': [], 'Q99527': [['1', 'N-acetylmethionine']], 'Q9BXC0': [], 'Q8TDS4': [['328', 'Phosphoserine']], 'P49019': [], 'Q96P68': [], 'Q9BXA5': [], 'P08100': [['1', 'N-acetylmethionine'], ['296', 'N6-(retinylidene)lysine'], ['334', 'Phosphoserine'], ['336', 'Phosphothreonine'], ['338', 'Phosphoserine'], ['340', 'Phosphothreonine'], ['342', 'Phosphothreonine'], ['343', 'Phosphoserine']], 'Q9UHM6': [['340', 'N6-(retinylidene)lysine"\n']], 'Q9H1Y3': [['299', 'N6-(retinylidene)lysine"\n']], 'Q6U736': [['296', 'N6-(retinylidene)lysine"\n']], 'P03999': [['293', 'N6-(retinylidene)lysine"\n']], 'P04001': [['312', 'N6-(retinylidene)lysine']], 'P04000': [['312', 'N6-(retinylidene)lysine"\n']], 'P46091': [], 'P46089': [['324', 'Phosphoserine'], ['326', 'Phosphoserine'], ['328', 'Phosphoserine']], 'P46093': [], 'P46095': [['356', 'Phosphoserine'], ['358', 'Phosphoserine'], ['360', 'Phosphoserine']], 'P47775': [['330', 'Phosphoserine'], ['332', 'Phosphoserine']], 'P49685': [], 'Q13304': [], 'Q15760': [], 'Q99678': [], 'Q99679': [], 'Q99680': [], 'O00155': [], 'Q8NDV2': [], 'Q9NS67': [], 'O00270': [], 'O75388': [], 'Q49SQ1': [], 'Q9UPC5': [], 'Q9HC97': [], 'O15354': [], 'O60883': [['471', 'Phosphoserine'], ['479', 'Phosphothreonine']], 'O43194': [['396', 'Phosphoserine']], 'Q9Y5Y3': [], 'Q13585': [], 'Q9Y2T5': [], 'Q9BZJ8': [], 'Q9BZJ7': [], 'Q9BZJ6': [], 'Q8IYL9': [], 'Q15743': [], 'O95800': [], 'Q96P69': [], 'Q96P67': [], 'Q9NYM4': [], 'Q9NQS5': [['221', 'Phosphoserine'], ['224', 'Phosphoserine']], 'P60893': [], 'Q9BY21': [], 'Q9GZN0': [], 'Q96P66': [], 'Q9UNW8': [], 'Q8IZ08': [], 'Q6DWJ6': [], 'Q7Z602': [], 'Q7Z601': [], 'Q96CH1': [], 'Q8TDV2': [], 'Q86SP6': [], 'Q8NGU9': [], 'Q8TDV0': [], 'Q8TDT2': [], 'Q6NV75': [], 'Q9UJ42': [], 'Q8N6U8': [], 'Q16538': [['413', 'Phosphoserine'], ['435', 'Phosphoserine']], 'O14626': [], 'Q9NS66': [], 'Q9BXC1': [], 'Q14439': [], 'O15218': [], 'P32249': [['328', 'Phosphoserine']], 'Q9BXB1': [['920', 'Phosphoserine']], 'O75473': [], 'Q9HBX8': [], 'P04201': [], 'P35410': [], 'Q8TDS7': [], 'Q86SM8': [], 'Q96AM1': [], 'Q86SM5': [], 'Q96LB2': [], 'Q96LB1': [], 'Q96LB0': [], 'Q96LA9': [], 'Q86VZ1': [], 'O00398': [], 'Q9P1P5': [], 'O14804': [], 'Q96RI8': [], 'Q969N4': [], 'Q96RI9': [], 'Q16602': [['420', 'Phosphoserine'], ['445', 'Phosphoserine']], 'Q13324': [], 'Q02643': [], 'P48546': [], 'P43220': [['341', 'ADP-ribosylcysteine'], ['348', 'ADP-ribosylarginine']], 'O95838': [], 'P47871': [['456', 'Phosphoserine'], ['459', 'Phosphoserine']], 'P47872': [], 'Q03431': [['551', 'Phosphothreonine']], 'P49190': [], 'P41586': [['434', 'Phosphoserine'], ['447', 'Phosphoserine']], 'P32241': [], 'P41587': [], 'O94910': [['1194', 'Omega-N-methylarginine'], ['1220', 'Phosphoserine'], ['1473', 'Phosphoserine']], 'O95490': [['1374', 'Phosphoserine'], ['1409', 'Phosphoserine'], ['1430', 'Phosphoserine']], 'Q9HAR2': [['1164', 'Phosphoserine']], 'Q9HBW9': [], 'Q14246': [], 'Q9UHX3': [], 'Q9BY15': [], 'P48960': [['815', 'Phosphoserine'], ['816', 'Phosphothreonine'], ['818', 'Phosphoserine'], ['825', 'Phosphothreonine'], ['831', 'Phosphoserine'], ['833', 'Phosphoserine']], 'Q86SQ6': [], 'Q96PE1': [['1107', 'Phosphoserine']], 'Q8IWK6': [], 'Q9NYQ6': [['1666', '(3R)-3-hydroxyasparagine'], ['1889', '(3R)-3-hydroxyaspartate'], ['2761', 'Phosphoserine'], ['2764', 'Phosphoserine'], ['2871', 'Phosphoserine'], ['2873', 'Phosphoserine']], 'Q9HCU4': [['1591', '(3R)-3-hydroxyasparagine'], ['1810', '(3R)-3-hydroxyasparagine']], 'Q9NYQ7': [['1963', '(3R)-3-hydroxyaspartate'], ['2126', 'Phosphotyrosine'], ['3051', 'Phosphotyrosine'], ['3097', 'Phosphoserine']], 'Q6QNK2': [], 'Q7Z7M1': [], 'Q5T601': [], 'Q8IZF7': [], 'Q8IZF5': [], 'Q8IZF3': [], 'Q8IZF2': [['1300', 'Phosphothreonine'], ['1307', 'Phosphoserine']], 'O14514': [['609', 'Phosphothreonine'], ['1469', 'Phosphoserine']], 'O60241': [['1351', 'Phosphotyrosine']], 'O60242': [['619', 'Phosphoserine'], ['1220', 'Phosphoserine'], ['1411', 'Phosphoserine']], 'Q9Y653': [], 'Q8IZP9': [['1010', 'Phosphoserine']], 'Q86Y34': [], 'Q8IZF6': [], 'Q8IZF4': [], 'Q86SQ4': [['1165', 'Phosphoserine'], ['1168', 'Phosphoserine']], 'Q96K78': [], 'Q8WXG9': [], 'P41180': [['920', 'Phosphoserine'], ['1061', 'Phosphoserine']], 'Q9UBS5': [['873', 'Phosphothreonine'], ['930', 'Phosphothreonine']], 'O75899': [['776', 'Phosphoserine'], ['779', 'Phosphoserine'], ['819', 'Phosphothreonine'], ['884', 'Phosphoserine'], ['893', 'Phosphoserine'], ['913', 'Phosphoserine'], ['916', 'Phosphoserine'], ['920', 'Phosphoserine'], ['924', 'Phosphoserine']], 'Q13255': [['853', 'Phosphoserine'], ['871', 'Phosphothreonine'], ['894', 'Phosphoserine'], ['969', 'Phosphoserine'], ['1091', 'Phosphoserine'], ['1142', 'Phosphoserine'], ['1146', 'Phosphothreonine'], ['1149', 'Phosphoserine']], 'Q14416': [], 'Q14832': [], 'Q14833': [], 'P41594': [['861', 'Phosphoserine'], ['869', 'Omega-N-methylarginine'], ['925', 'Omega-N-methylarginine'], ['1018', 'Phosphoserine'], ['1020', 'Phosphoserine']], 'O15303': [], 'Q14831': [['900', 'Phosphoserine']], 'O00222': [], 'Q7RTX1': [], 'Q8TE23': [], 'Q7RTX0': [], 'Q8NFN8': [], 'Q5T848': [['694', 'Phosphoserine'], ['705', 'Phosphoserine'], ['708', 'Phosphoserine'], ['865', 'Phosphoserine'], ['946', 'Phosphoserine'], ['1080', 'Phosphoserine']], 'Q6PRD1': [], 'Q8NFJ5': [['301', 'Phosphoserine'], ['317', 'Phosphotyrosine'], ['320', 'Phosphotyrosine'], ['345', 'Phosphoserine'], ['347', 'Phosphotyrosine'], ['350', 'Phosphotyrosine']], 'Q9NZH0': [['354', 'Phosphoserine']], 'Q9NQ84': [['344', 'Phosphoserine'], ['383', 'Phosphoserine'], ['403', 'Phosphoserine'], ['406', 'Phosphoserine'], ['414', 'Phosphotyrosine'], ['423', 'Phosphothreonine']], 'Q9NZD1': [], 'Q5T6X5': [], 'Q9UP38': [], 'Q14332': [], 'Q9NPG1': [], 'Q9ULV1': [], 'Q13467': [], 'O60353': [['653', 'Phosphoserine']], 'O75084': [], 'Q9H461': [], 'O00144': [], 'Q9ULW2': [], 'Q9NYW7': [], 'Q9NYW6': [], 'Q9NYW5': [], 'Q9NYW4': [], 'Q9NYW3': [], 'Q9NYW2': [], 'Q9NYW1': [], 'Q9NYW0': [], 'Q9NYV9': [], 'Q9NYV8': [], 'Q9NYV7': [], 'P59542': [], 'P59543': [], 'P59541': [], 'P59538': [], 'P59533': [], 'P59534': [], 'P59535': [], 'P59536': [], 'Q7RTR8': [], 'P59537': [], 'P59539': [], 'P59540': [], 'P59544': [], 'P59551': [], 'Q5VW38': [], 'Q96N19': [], 'Q8TCB6': [], 'Q86W33': [], 'P51810': [], 'Q5UAW9': []}


## Glycosylation
Covalently attached glycan group(s)

In [5]:
# Build the dictionary keys as the entry names, values are lists [position, ptmtype]
dicPTMglyco=dict_ptm(os.path.join(resultspath,'remaining3/myprot_list_struc_remaining_merged_ENSG.csv'),'glyco')

P08100
Reponse from uniprot: Glycosylation
CARBOHYD 2;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000250|UniProtKB:P02699"; CARBOHYD 15;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000269|PubMed:28753425"

res: [['2', 'N-linked (GlcNAc...) asparagine'], ['15', 'N-linked (GlcNAc...) asparagine']]
Q9UHM6
Reponse from uniprot: Glycosylation


res: []
Q9H1Y3
Reponse from uniprot: Glycosylation
CARBOHYD 5;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000255"; CARBOHYD 198;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000255"

res: [['5', 'N-linked (GlcNAc...) asparagine'], ['198', 'N-linked (GlcNAc...) asparagine']]
Q6U736
Reponse from uniprot: Glycosylation
CARBOHYD 4;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000255"

res: [['4', 'N-linked (GlcNAc...) asparagine']]
P03999
Reponse from uniprot: Glycosylation
CARBOHYD 14;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000305"

res: [['14', 'N-linked (Glc

In [27]:
# avoid downloading it again
dicPTMglyco={'P18089': [], 'P21917': [['3', 'N-linked (GlcNAc...) asparagine']], 'P35367': [['5', 'N-linked (GlcNAc...) asparagine'], ['18', 'N-linked (GlcNAc...) asparagine']], 'P48145': [['3', 'N-linked (GlcNAc...) asparagine'], ['13', 'N-linked (GlcNAc...) asparagine'], ['25', 'N-linked (GlcNAc...) asparagine']], 'P48146': [['24', 'N-linked (GlcNAc...) asparagine'], ['29', 'N-linked (GlcNAc...) asparagine'], ['34', 'N-linked (GlcNAc...) asparagine']], 'Q99463': [['11', 'N-linked (GlcNAc...) asparagine']], 'Q9NSD7': [['36', 'N-linked (GlcNAc...) asparagine'], ['40', 'N-linked (GlcNAc...) asparagine']], 'P32745': [['17', 'N-linked (GlcNAc...) asparagine'], ['30', 'N-linked (GlcNAc...) asparagine']], 'Q15722': [['2', 'N-linked (GlcNAc...) asparagine'], ['164', 'N-linked (GlcNAc...) asparagine']], 'Q9NPC1': [['10', 'N-linked (GlcNAc...) asparagine']], 'P0DMS8': [['3', 'N-linked (GlcNAc...) asparagine'], ['4', 'N-linked (GlcNAc...) asparagine'], ['12', 'N-linked (GlcNAc...) asparagine']], 'P46091': [['14', 'N-linked (GlcNAc...) asparagine']], 'Q99678': [['16', 'N-linked (GlcNAc...) asparagine'], ['26', 'N-linked (GlcNAc...) asparagine']], 'Q99680': [['16', 'N-linked (GlcNAc...) asparagine'], ['192', 'N-linked (GlcNAc...) asparagine']], 'Q96CH1': [['8', 'N-linked (GlcNAc...) asparagine']], 'Q8TDV2': [['34', 'N-linked (GlcNAc...) asparagine']], 'P35410': [['54', 'N-linked (GlcNAc...) asparagine'], ['57', 'N-linked (GlcNAc...) asparagine']], 'Q9P1P4': [['145', 'N-linked (GlcNAc...) asparagine']], 'O00398': [['16', 'N-linked (GlcNAc...) asparagine'], ['26', 'N-linked (GlcNAc...) asparagine'], ['178', 'N-linked (GlcNAc...) asparagine']], 'Q9P1P5': [['24', 'N-linked (GlcNAc...) asparagine'], ['30', 'N-linked (GlcNAc...) asparagine'], ['289', 'N-linked (GlcNAc...) asparagine']], 'O14804': [['21', 'N-linked (GlcNAc...) asparagine']], 'Q96RI8': [['4', 'N-linked (GlcNAc...) asparagine'], ['19', 'N-linked (GlcNAc...) asparagine']], 'Q969N4': [['4', 'N-linked (GlcNAc...) asparagine'], ['18', 'N-linked (GlcNAc...) asparagine']], 'Q96RI9': [['4', 'N-linked (GlcNAc...) asparagine'], ['19', 'N-linked (GlcNAc...) asparagine']]}

# old {'P08908': [['10', 'N-linked (GlcNAc...) asparagine'], ['11', 'N-linked (GlcNAc...) asparagine'], ['24', 'N-linked (GlcNAc...) asparagine']], 'P28222': [['24', 'N-linked (GlcNAc...) asparagine'], ['32', 'N-linked (GlcNAc...) asparagine']], 'P28221': [['5', 'N-linked (GlcNAc...) asparagine'], ['17', 'N-linked (GlcNAc...) asparagine'], ['21', 'N-linked (GlcNAc...) asparagine']], 'P28566': [['2', 'N-linked (GlcNAc...) asparagine'], ['5', 'N-linked (GlcNAc...) asparagine']], 'P30939': [['5', 'N-linked (GlcNAc...) asparagine'], ['10', 'N-linked (GlcNAc...) asparagine']], 'P28223': [['8', 'N-linked (GlcNAc...) asparagine'], ['38', 'N-linked (GlcNAc...) asparagine'], ['44', 'N-linked (GlcNAc...) asparagine'], ['51', 'N-linked (GlcNAc...) asparagine'], ['54', 'N-linked (GlcNAc...) asparagine']], 'P41595': [['30', 'N-linked (GlcNAc...) asparagine']], 'P28335': [['39', 'N-linked (GlcNAc...) asparagine']], 'Q13639': [['7', 'N-linked (GlcNAc...) asparagine']], 'P47898': [['6', 'N-linked (GlcNAc...) asparagine'], ['21', 'N-linked (GlcNAc...) asparagine']], 'P50406': [], 'P34969': [['5', 'N-linked (GlcNAc...) asparagine'], ['66', 'N-linked (GlcNAc...) asparagine']], 'P11229': [['2', 'N-linked (GlcNAc...) asparagine'], ['12', 'N-linked (GlcNAc...) asparagine']], 'P08172': [['2', 'N-linked (GlcNAc...) asparagine'], ['3', 'N-linked (GlcNAc...) asparagine'], ['6', 'N-linked (GlcNAc...) asparagine'], ['9', 'N-linked (GlcNAc...) asparagine']], 'P20309': [['5', 'N-linked (GlcNAc...) asparagine'], ['6', 'N-linked (GlcNAc...) asparagine'], ['15', 'N-linked (GlcNAc...) asparagine'], ['41', 'N-linked (GlcNAc...) asparagine'], ['48', 'N-linked (GlcNAc...) asparagine']], 'P08173': [['8', 'N-linked (GlcNAc...) asparagine'], ['13', 'N-linked (GlcNAc...) asparagine']], 'P08912': [['8', 'N-linked (GlcNAc...) asparagine']], 'P35348': [['7', 'N-linked (GlcNAc...) asparagine'], ['13', 'N-linked (GlcNAc...) asparagine'], ['22', 'N-linked (GlcNAc...) asparagine']], 'P35368': [['10', 'N-linked (GlcNAc...) asparagine'], ['24', 'N-linked (GlcNAc...) asparagine'], ['29', 'N-linked (GlcNAc...) asparagine'], ['34', 'N-linked (GlcNAc...) asparagine']], 'P25100': [['65', 'N-linked (GlcNAc...) asparagine'], ['82', 'N-linked (GlcNAc...) asparagine']], 'P08913': [['25', 'N-linked (GlcNAc...) asparagine'], ['29', 'N-linked (GlcNAc...) asparagine']], 'P18089': [], 'P18825': [['19', 'N-linked (GlcNAc...) asparagine'], ['33', 'N-linked (GlcNAc...) asparagine']], 'P08588': [['15', 'N-linked (GlcNAc...) asparagine']], 'P07550': [['6', 'N-linked (GlcNAc...) asparagine'], ['15', 'N-linked (GlcNAc...) asparagine']], 'P13945': [['8', 'N-linked (GlcNAc...) asparagine'], ['26', 'N-linked (GlcNAc...) asparagine']], 'P21728': [['5', 'N-linked (GlcNAc...) asparagine']], 'P14416': [['5', 'N-linked (GlcNAc...) asparagine'], ['17', 'N-linked (GlcNAc...) asparagine'], ['23', 'N-linked (GlcNAc...) asparagine']], 'P35462': [['12', 'N-linked (GlcNAc...) asparagine'], ['19', 'N-linked (GlcNAc...) asparagine'], ['97', 'N-linked (GlcNAc...) asparagine'], ['173', 'N-linked (GlcNAc...) asparagine']], 'P21917': [['3', 'N-linked (GlcNAc...) asparagine']], 'P21918': [['7', 'N-linked (GlcNAc...) asparagine'], ['222', 'N-linked (GlcNAc...) asparagine']], 'P35367': [['5', 'N-linked (GlcNAc...) asparagine'], ['18', 'N-linked (GlcNAc...) asparagine']], 'P25021': [['4', 'N-linked (GlcNAc...) asparagine']], 'Q9Y5N1': [['11', 'N-linked (GlcNAc...) asparagine']], 'Q9H3N8': [['5', 'N-linked (GlcNAc...) asparagine'], ['9', 'N-linked (GlcNAc...) asparagine']], 'Q96RJ0': [['10', 'N-linked (GlcNAc...) asparagine'], ['17', 'N-linked (GlcNAc...) asparagine']], 'P30556': [['4', 'N-linked (GlcNAc...) (complex) asparagine'], ['176', 'N-linked (GlcNAc...) asparagine'], ['188', 'N-linked (GlcNAc...) asparagine']], 'P50052': [['4', 'N-linked (GlcNAc...) asparagine'], ['13', 'N-linked (GlcNAc...) asparagine'], ['24', 'N-linked (GlcNAc...) asparagine'], ['29', 'N-linked (GlcNAc...) asparagine'], ['34', 'N-linked (GlcNAc...) asparagine']], 'P35414': [['15', 'N-linked (GlcNAc...) asparagine'], ['175', 'N-linked (GlcNAc...) asparagine']], 'P28336': [['8', 'N-linked (GlcNAc...) asparagine'], ['16', 'N-linked (GlcNAc...) asparagine'], ['192', 'N-linked (GlcNAc...) asparagine']], 'P30550': [['20', 'N-linked (GlcNAc...) asparagine']], 'P32247': [['10', 'N-linked (GlcNAc...) asparagine'], ['18', 'N-linked (GlcNAc...) asparagine']], 'P46663': [['14', 'N-linked (GlcNAc...) asparagine'], ['22', 'N-linked (GlcNAc...) asparagine'], ['185', 'N-linked (GlcNAc...) asparagine']], 'P30411': [['30', 'N-linked (GlcNAc...) asparagine'], ['39', 'N-linked (GlcNAc...) asparagine'], ['207', 'N-linked (GlcNAc...) asparagine']], 'P32238': [['10', 'N-linked (GlcNAc...) asparagine'], ['24', 'N-linked (GlcNAc...) asparagine'], ['190', 'N-linked (GlcNAc...) asparagine']], 'P32239': [['7', 'N-linked (GlcNAc...) asparagine'], ['30', 'N-linked (GlcNAc...) asparagine'], ['36', 'N-linked (GlcNAc...) asparagine']], 'Q16581': [['9', 'N-linked (GlcNAc...) asparagine'], ['194', 'N-linked (GlcNAc...) asparagine'], ['266', 'O-linked (GalNAc...) serine']], 'P21730': [['5', 'N-linked (GlcNAc...) asparagine']], 'Q9P296': [['3', 'N-linked (GlcNAc...) asparagine']], 'P25101': [['29', 'N-linked (GlcNAc...) asparagine'], ['62', 'N-linked (GlcNAc...) asparagine']], 'P24530': [['59', 'N-linked (GlcNAc...) asparagine']], 'P21462': [['4', 'N-linked (GlcNAc...) asparagine'], ['10', 'N-linked (GlcNAc...) asparagine']], 'P25090': [['4', 'N-linked (GlcNAc...) asparagine']], 'P25089': [['4', 'N-linked (GlcNAc...) asparagine'], ['10', 'N-linked (GlcNAc...) asparagine']], 'P47211': [['7', 'N-linked (GlcNAc...) asparagine'], ['12', 'N-linked (GlcNAc...) asparagine'], ['183', 'N-linked (GlcNAc...) asparagine']], 'O43603': [['2', 'N-linked (GlcNAc...) asparagine'], ['11', 'N-linked (GlcNAc...) asparagine']], 'O60755': [['6', 'N-linked (GlcNAc...) asparagine']], 'Q92847': [['13', 'N-linked (GlcNAc...) asparagine'], ['27', 'N-linked (GlcNAc...) asparagine']], 'P30968': [['18', 'N-linked (GlcNAc...) asparagine'], ['102', 'N-linked (GlcNAc...) asparagine']], 'Q969F8': [['10', 'N-linked (GlcNAc...) asparagine'], ['18', 'N-linked (GlcNAc...) asparagine'], ['28', 'N-linked (GlcNAc...) asparagine']], 'Q99705': [['82', 'N-linked (GlcNAc...) asparagine'], ['85', 'N-linked (GlcNAc...) asparagine'], ['92', 'N-linked (GlcNAc...) asparagine']], 'Q969V1': [['10', 'N-linked (GlcNAc...) asparagine'], ['17', 'N-linked (GlcNAc...) asparagine']], 'Q01726': [['29', 'N-linked (GlcNAc...) asparagine']], 'Q01718': [['12', 'N-linked (GlcNAc...) asparagine'], ['17', 'N-linked (GlcNAc...) asparagine']], 'P41968': [['2', 'N-linked (GlcNAc...) asparagine'], ['16', 'N-linked (GlcNAc...) asparagine'], ['28', 'N-linked (GlcNAc...) asparagine']], 'P32245': [['3', 'N-linked (GlcNAc...) asparagine'], ['17', 'N-linked (GlcNAc...) asparagine'], ['26', 'N-linked (GlcNAc...) asparagine']], 'P33032': [['2', 'N-linked (GlcNAc...) asparagine'], ['15', 'N-linked (GlcNAc...) asparagine'], ['20', 'N-linked (GlcNAc...) asparagine'], ['28', 'N-linked (GlcNAc...) asparagine']], 'O43193': [['6', 'N-linked (GlcNAc...) asparagine'], ['192', 'N-linked (GlcNAc...) asparagine']], 'Q9HB89': [['7', 'N-linked (GlcNAc...) asparagine'], ['27', 'N-linked (GlcNAc...) asparagine'], ['41', 'N-linked (GlcNAc...) asparagine']], 'Q9GZQ4': [['9', 'N-linked (GlcNAc...) asparagine'], ['27', 'N-linked (GlcNAc...) asparagine'], ['194', 'N-linked (GlcNAc...) asparagine']], 'Q9GZQ6': [['10', 'N-linked (GlcNAc...) asparagine'], ['18', 'N-linked (GlcNAc...) asparagine'], ['29', 'N-linked (GlcNAc...) asparagine'], ['113', 'N-linked (GlcNAc...) asparagine'], ['195', 'N-linked (GlcNAc...) asparagine']], 'Q9Y5X5': [['110', 'N-linked (GlcNAc...) asparagine'], ['122', 'N-linked (GlcNAc...) asparagine'], ['133', 'N-linked (GlcNAc...) asparagine'], ['300', 'N-linked (GlcNAc...) asparagine']], 'Q6W5P4': [['4', 'N-linked (GlcNAc...) asparagine']], 'P48145': [['3', 'N-linked (GlcNAc...) asparagine'], ['13', 'N-linked (GlcNAc...) asparagine'], ['25', 'N-linked (GlcNAc...) asparagine']], 'P48146': [['24', 'N-linked (GlcNAc...) asparagine'], ['29', 'N-linked (GlcNAc...) asparagine'], ['34', 'N-linked (GlcNAc...) asparagine']], 'P25929': [['2', 'N-linked (GlcNAc...) asparagine'], ['11', 'N-linked (GlcNAc...) asparagine'], ['17', 'N-linked (GlcNAc...) asparagine']], 'P49146': [['11', 'N-linked (GlcNAc...) asparagine']], 'P50391': [['2', 'N-linked (GlcNAc...) asparagine'], ['19', 'N-linked (GlcNAc...) asparagine'], ['29', 'N-linked (GlcNAc...) asparagine'], ['187', 'N-linked (GlcNAc...) asparagine']], 'Q15761': [['10', 'N-linked (GlcNAc...) asparagine'], ['17', 'N-linked (GlcNAc...) asparagine']], 'Q99463': [['11', 'N-linked (GlcNAc...) asparagine']], 'P30989': [['4', 'N-linked (GlcNAc...) asparagine'], ['37', 'N-linked (GlcNAc...) asparagine'], ['41', 'N-linked (GlcNAc...) asparagine']], 'O95665': [], 'P41143': [['18', 'N-linked (GlcNAc...) asparagine'], ['33', 'N-linked (GlcNAc...) asparagine']], 'P41145': [['25', 'N-linked (GlcNAc...) asparagine'], ['39', 'N-linked (GlcNAc...) asparagine']], 'P35372': [['9', 'N-linked (GlcNAc...) asparagine'], ['12', 'N-linked (GlcNAc...) asparagine'], ['33', 'N-linked (GlcNAc...) asparagine'], ['40', 'N-linked (GlcNAc...) asparagine'], ['48', 'N-linked (GlcNAc...) asparagine']], 'P41146': [['21', 'N-linked (GlcNAc...) asparagine'], ['28', 'N-linked (GlcNAc...) asparagine'], ['39', 'N-linked (GlcNAc...) asparagine']], 'O43613': [['194', 'N-linked (GlcNAc...) asparagine']], 'O43614': [['14', 'N-linked (GlcNAc...) asparagine'], ['22', 'N-linked (GlcNAc...) asparagine'], ['202', 'N-linked (GlcNAc...) asparagine']], 'Q96P65': [['19', 'N-linked (GlcNAc...) asparagine']], 'P49683': [['27', 'N-linked (GlcNAc...) asparagine'], ['36', 'N-linked (GlcNAc...) asparagine']], 'P25116': [['35', 'N-linked (GlcNAc...) asparagine'], ['62', 'N-linked (GlcNAc...) asparagine'], ['75', 'N-linked (GlcNAc...) asparagine'], ['250', 'N-linked (GlcNAc...) asparagine'], ['259', 'N-linked (GlcNAc...) asparagine']], 'P55085': [['30', 'N-linked (GlcNAc...) asparagine'], ['222', 'N-linked (GlcNAc...) asparagine']], 'O00254': [['25', 'N-linked (GlcNAc...) asparagine'], ['82', 'N-linked (GlcNAc...) asparagine'], ['331', 'N-linked (GlcNAc...) asparagine']], 'Q96RI0': [['56', 'N-linked (GlcNAc...) asparagine']], 'Q9HBX9': [['36', 'N-linked (GlcNAc...) asparagine'], ['127', 'N-linked (GlcNAc...) asparagine'], ['264', 'N-linked (GlcNAc...) asparagine'], ['272', 'N-linked (GlcNAc...) asparagine'], ['325', 'N-linked (GlcNAc...) asparagine'], ['368', 'N-linked (GlcNAc...) asparagine']], 'Q8WXD0': [['54', 'N-linked (GlcNAc...) asparagine'], ['138', 'N-linked (GlcNAc...) asparagine'], ['274', 'N-linked (GlcNAc...) asparagine'], ['335', 'N-linked (GlcNAc...) asparagine'], ['378', 'N-linked (GlcNAc...) asparagine']], 'Q9NSD7': [['36', 'N-linked (GlcNAc...) asparagine'], ['40', 'N-linked (GlcNAc...) asparagine']], 'Q8TDU9': [['5', 'N-linked (GlcNAc...) asparagine'], ['17', 'N-linked (GlcNAc...) asparagine']], 'P30872': [['4', 'N-linked (GlcNAc...) asparagine'], ['44', 'N-linked (GlcNAc...) asparagine'], ['48', 'N-linked (GlcNAc...) asparagine']], 'P30874': [['9', 'N-linked (GlcNAc...) asparagine'], ['22', 'N-linked (GlcNAc...) asparagine'], ['29', 'N-linked (GlcNAc...) asparagine'], ['32', 'N-linked (GlcNAc...) asparagine']], 'P32745': [['17', 'N-linked (GlcNAc...) asparagine'], ['30', 'N-linked (GlcNAc...) asparagine']], 'P31391': [['24', 'N-linked (GlcNAc...) asparagine']], 'P35346': [['13', 'N-linked (GlcNAc...) asparagine'], ['26', 'N-linked (GlcNAc...) asparagine'], ['187', 'N-linked (GlcNAc...) asparagine']], 'P25103': [['14', 'N-linked (GlcNAc...) asparagine'], ['18', 'N-linked (GlcNAc...) asparagine']], 'P21452': [['11', 'N-linked (GlcNAc...) asparagine'], ['19', 'N-linked (GlcNAc...) asparagine']], 'P29371': [['23', 'N-linked (GlcNAc...) asparagine'], ['50', 'N-linked (GlcNAc...) asparagine'], ['73', 'N-linked (GlcNAc...) asparagine']], 'P34981': [['3', 'N-linked (GlcNAc...) asparagine'], ['10', 'N-linked (GlcNAc...) asparagine']], 'Q9UKP6': [['29', 'N-linked (GlcNAc...) asparagine'], ['33', 'N-linked (GlcNAc...) asparagine']], 'P37288': [['27', 'N-linked (GlcNAc...) asparagine'], ['196', 'N-linked (GlcNAc...) asparagine']], 'P47901': [['21', 'N-linked (GlcNAc...) asparagine']], 'P30518': [['22', 'N-linked (GlcNAc...) asparagine']], 'P30559': [['8', 'N-linked (GlcNAc...) asparagine'], ['15', 'N-linked (GlcNAc...) asparagine'], ['26', 'N-linked (GlcNAc...) asparagine']], 'Q99788': [['9', 'N-linked (GlcNAc...) asparagine'], ['192', 'N-linked (GlcNAc...) asparagine']], 'P32246': [['5', 'N-linked (GlcNAc...) asparagine']], 'P41597': [['14', 'N-linked (GlcNAc...) asparagine']], 'P51677': [], 'P51679': [['183', 'N-linked (GlcNAc...) asparagine'], ['194', 'N-linked (GlcNAc...) asparagine']], 'P51681': [['6', 'O-linked (GalNAc...) serine'], ['7', 'O-linked (GalNAc...) serine'], ['16', 'O-linked (GalNAc...) threonine'], ['17', 'O-linked (GalNAc...) serine']], 'P51684': [['7', 'N-linked (GlcNAc...) asparagine'], ['23', 'N-linked (GlcNAc...) asparagine']], 'P32248': [['36', 'N-linked (GlcNAc...) asparagine']], 'P51685': [], 'P51686': [['32', 'N-linked (GlcNAc...) asparagine']], 'P46092': [], 'P25024': [['3', 'N-linked (GlcNAc...) asparagine'], ['16', 'N-linked (GlcNAc...) asparagine']], 'P25025': [['22', 'N-linked (GlcNAc...) asparagine']], 'P49682': [['22', 'N-linked (GlcNAc...) asparagine'], ['32', 'N-linked (GlcNAc...) asparagine']], 'P61073': [['11', 'N-linked (GlcNAc...) asparagine'], ['18', 'O-linked (Xyl...) (chondroitin sulfate) serine'], ['176', 'N-linked (GlcNAc...) asparagine']], 'P32302': [['28', 'N-linked (GlcNAc...) asparagine'], ['196', 'N-linked (GlcNAc...) asparagine']], 'O00574': [['16', 'N-linked (GlcNAc...) asparagine']], 'P49238': [], 'P46094': [], 'Q16570': [['16', 'N-linked (GlcNAc...) asparagine'], ['33', 'N-linked (GlcNAc...) asparagine']], 'O00590': [['19', 'N-linked (GlcNAc...) asparagine']], 'P25106': [['13', 'N-linked (GlcNAc...) asparagine'], ['22', 'N-linked (GlcNAc...) asparagine'], ['39', 'N-linked (GlcNAc...) asparagine']], 'Q9NPB9': [['6', 'N-linked (GlcNAc...) asparagine'], ['19', 'N-linked (GlcNAc...) asparagine']], 'O00421': [['3', 'N-linked (GlcNAc...) asparagine']], 'P23945': [['191', 'N-linked (GlcNAc...) asparagine'], ['199', 'N-linked (GlcNAc...) asparagine'], ['293', 'N-linked (GlcNAc...) asparagine'], ['318', 'N-linked (GlcNAc...) asparagine']], 'P22888': [['99', 'N-linked (GlcNAc...) asparagine'], ['174', 'N-linked (GlcNAc...) asparagine'], ['195', 'N-linked (GlcNAc...) asparagine'], ['291', 'N-linked (GlcNAc...) asparagine'], ['299', 'N-linked (GlcNAc...) asparagine'], ['313', 'N-linked (GlcNAc...) asparagine']], 'P16473': [['77', 'N-linked (GlcNAc...) asparagine'], ['99', 'N-linked (GlcNAc...) asparagine'], ['113', 'N-linked (GlcNAc...) asparagine'], ['177', 'N-linked (GlcNAc...) asparagine'], ['198', 'N-linked (GlcNAc...) asparagine'], ['302', 'N-linked (GlcNAc...) asparagine']], 'Q8TCW9': [['11', 'N-linked (GlcNAc...) asparagine'], ['14', 'N-linked (GlcNAc...) asparagine'], ['36', 'N-linked (GlcNAc...) asparagine']], 'Q8NFJ6': [['7', 'N-linked (GlcNAc...) asparagine'], ['27', 'N-linked (GlcNAc...) asparagine']], 'O14842': [['155', 'N-linked (GlcNAc...) asparagine']], 'O15552': [['151', 'N-linked (GlcNAc...) asparagine'], ['167', 'N-linked (GlcNAc...) asparagine']], 'O14843': [['166', 'N-linked (GlcNAc...) asparagine']], 'Q5NUL3': [['21', 'N-linked (GlcNAc...) asparagine']], 'O15529': [['166', 'N-linked (GlcNAc...) asparagine']], 'Q15722': [['2', 'N-linked (GlcNAc...) asparagine'], ['164', 'N-linked (GlcNAc...) asparagine']], 'Q9NPC1': [['10', 'N-linked (GlcNAc...) asparagine']], 'Q9Y271': [['6', 'N-linked (GlcNAc...) asparagine'], ['169', 'N-linked (GlcNAc...) asparagine'], ['180', 'N-linked (GlcNAc...) asparagine'], ['262', 'N-linked (GlcNAc...) asparagine']], 'Q9NS75': [['20', 'N-linked (GlcNAc...) asparagine'], ['26', 'N-linked (GlcNAc...) asparagine'], ['30', 'N-linked (GlcNAc...) asparagine'], ['181', 'N-linked (GlcNAc...) asparagine']], 'Q8TDS5': [['44', 'N-linked (GlcNAc...) asparagine']], 'Q92633': [['27', 'N-linked (GlcNAc...) asparagine'], ['35', 'N-linked (GlcNAc...) asparagine']], 'Q9HBW0': [['10', 'N-linked (GlcNAc...) asparagine'], ['18', 'N-linked (GlcNAc...) asparagine']], 'Q9UBY5': [['15', 'N-linked (GlcNAc...) asparagine'], ['172', 'N-linked (GlcNAc...) asparagine']], 'Q99677': [['15', 'N-linked (GlcNAc...) asparagine'], ['24', 'N-linked (GlcNAc...) asparagine'], ['28', 'N-linked (GlcNAc...) asparagine'], ['183', 'N-linked (GlcNAc...) asparagine']], 'Q9H1C0': [['4', 'N-linked (GlcNAc...) asparagine'], ['9', 'N-linked (GlcNAc...) asparagine']], 'P43657': [['5', 'N-linked (GlcNAc...) asparagine']], 'P21453': [['30', 'N-linked (GlcNAc...) asparagine'], ['36', 'N-linked (GlcNAc...) asparagine']], 'O95136': [['19', 'N-linked (GlcNAc...) asparagine']], 'Q99500': [['15', 'N-linked (GlcNAc...) asparagine']], 'O95977': [['2', 'N-linked (GlcNAc...) asparagine'], ['30', 'N-linked (GlcNAc...) asparagine']], 'Q9H228': [['20', 'N-linked (GlcNAc...) asparagine']], 'P21554': [['77', 'N-linked (GlcNAc...) asparagine'], ['83', 'N-linked (GlcNAc...) asparagine']], 'P34972': [['11', 'N-linked (GlcNAc...) asparagine']], 'Q14330': [['14', 'N-linked (GlcNAc...) asparagine']], 'Q9Y2T6': [['5', 'N-linked (GlcNAc...) asparagine'], ['171', 'N-linked (GlcNAc...) asparagine']], 'Q8TDV5': [], 'P25105': [['169', 'N-linked (GlcNAc...) asparagine']], 'Q13258': [['10', 'N-linked (GlcNAc...) asparagine'], ['90', 'N-linked (GlcNAc...) asparagine'], ['297', 'N-linked (GlcNAc...) asparagine']], 'Q9Y5Y4': [['4', 'N-linked (GlcNAc...) asparagine'], ['25', 'N-linked (GlcNAc...) asparagine']], 'P34995': [['8', 'N-linked (GlcNAc...) asparagine'], ['25', 'N-linked (GlcNAc...) asparagine']], 'P43116': [['3', 'N-linked (GlcNAc...) asparagine'], ['6', 'N-linked (GlcNAc...) asparagine'], ['96', 'N-linked (GlcNAc...) asparagine'], ['287', 'N-linked (GlcNAc...) asparagine']], 'P43115': [['18', 'N-linked (GlcNAc...) asparagine'], ['36', 'N-linked (GlcNAc...) asparagine']], 'P35408': [['7', 'N-linked (GlcNAc...) asparagine']], 'P43088': [['4', 'N-linked (GlcNAc...) asparagine'], ['19', 'N-linked (GlcNAc...) asparagine']], 'P43119': [['7', 'N-linked (GlcNAc...) asparagine']], 'P21731': [['4', 'N-linked (GlcNAc...) asparagine'], ['16', 'N-linked (GlcNAc...) asparagine"\n']], 'P48039': [['4', 'N-linked (GlcNAc...) asparagine'], ['10', 'N-linked (GlcNAc...) asparagine']], 'P49286': [['4', 'N-linked (GlcNAc...) asparagine']], 'P30542': [['159', 'N-linked (GlcNAc...) asparagine']], 'P29274': [['154', 'N-linked (GlcNAc...) asparagine']], 'P29275': [['153', 'N-linked (GlcNAc...) asparagine'], ['163', 'N-linked (GlcNAc...) asparagine']], 'P0DMS8': [['3', 'N-linked (GlcNAc...) asparagine'], ['4', 'N-linked (GlcNAc...) asparagine'], ['12', 'N-linked (GlcNAc...) asparagine']], 'P47900': [['11', 'N-linked (GlcNAc...) asparagine'], ['27', 'N-linked (GlcNAc...) asparagine'], ['113', 'N-linked (GlcNAc...) asparagine'], ['197', 'N-linked (GlcNAc...) asparagine']], 'P41231': [['9', 'N-linked (GlcNAc...) asparagine'], ['13', 'N-linked (GlcNAc...) asparagine']], 'P51582': [], 'Q15077': [['5', 'N-linked (GlcNAc...) asparagine']], 'Q96G91': [['4', 'N-linked (GlcNAc...) asparagine'], ['179', 'N-linked (GlcNAc...) asparagine']], 'Q9H244': [['6', 'N-linked (GlcNAc...) asparagine'], ['13', 'N-linked (GlcNAc...) asparagine']], 'Q9BPV8': [['23', 'N-linked (GlcNAc...) asparagine'], ['31', 'N-linked (GlcNAc...) asparagine'], ['285', 'N-linked (GlcNAc...) asparagine']], 'Q15391': [['3', 'N-linked (GlcNAc...) asparagine'], ['161', 'N-linked (GlcNAc...) asparagine']], 'Q8TDU6': [['4', 'N-linked (GlcNAc...) asparagine'], ['76', 'N-linked (GlcNAc...) asparagine']], 'Q99527': [['25', 'N-linked (GlcNAc...) asparagine'], ['32', 'N-linked (GlcNAc...) asparagine'], ['44', 'N-linked (GlcNAc...) asparagine']], 'Q9BXC0': [['3', 'N-linked (GlcNAc...) asparagine']], 'Q8TDS4': [], 'P49019': [], 'Q96P68': [['10', 'N-linked (GlcNAc...) asparagine'], ['23', 'N-linked (GlcNAc...) asparagine'], ['176', 'N-linked (GlcNAc...) asparagine'], ['179', 'N-linked (GlcNAc...) asparagine']], 'Q9BXA5': [['8', 'N-linked (GlcNAc...) asparagine'], ['168', 'N-linked (GlcNAc...) asparagine']], 'P08100': [['2', 'N-linked (GlcNAc...) asparagine'], ['15', 'N-linked (GlcNAc...) asparagine']], 'Q9UHM6': [], 'Q9H1Y3': [['5', 'N-linked (GlcNAc...) asparagine'], ['198', 'N-linked (GlcNAc...) asparagine']], 'Q6U736': [['4', 'N-linked (GlcNAc...) asparagine']], 'P03999': [['14', 'N-linked (GlcNAc...) asparagine']], 'P04001': [['34', 'N-linked (GlcNAc...) asparagine']], 'P04000': [['22', 'O-linked (GlcNAc) serine'], ['34', 'N-linked (GlcNAc...) asparagine']], 'P46091': [['14', 'N-linked (GlcNAc...) asparagine']], 'P46089': [['20', 'N-linked (GlcNAc...) asparagine']], 'P46093': [['3', 'N-linked (GlcNAc...) asparagine'], ['164', 'N-linked (GlcNAc...) asparagine']], 'P46095': [['2', 'N-linked (GlcNAc...) asparagine'], ['9', 'N-linked (GlcNAc...) asparagine'], ['51', 'N-linked (GlcNAc...) asparagine']], 'P47775': [['8', 'N-linked (GlcNAc...) asparagine'], ['24', 'N-linked (GlcNAc...) asparagine']], 'P49685': [], 'Q13304': [['42', 'N-linked (GlcNAc...) asparagine'], ['204', 'N-linked (GlcNAc...) asparagine'], ['282', 'N-linked (GlcNAc...) asparagine']], 'Q15760': [['25', 'N-linked (GlcNAc...) asparagine'], ['52', 'N-linked (GlcNAc...) asparagine']], 'Q99678': [['16', 'N-linked (GlcNAc...) asparagine'], ['26', 'N-linked (GlcNAc...) asparagine']], 'Q99679': [['2', 'N-linked (GlcNAc...) asparagine'], ['8', 'N-linked (GlcNAc...) asparagine']], 'Q99680': [['16', 'N-linked (GlcNAc...) asparagine'], ['192', 'N-linked (GlcNAc...) asparagine']], 'O00155': [], 'Q8NDV2': [], 'Q9NS67': [['3', 'N-linked (GlcNAc...) asparagine']], 'O00270': [['5', 'N-linked (GlcNAc...) asparagine']], 'O75388': [['30', 'N-linked (GlcNAc...) asparagine'], ['199', 'N-linked (GlcNAc...) asparagine']], 'Q49SQ1': [['5', 'N-linked (GlcNAc...) asparagine'], ['12', 'N-linked (GlcNAc...) asparagine'], ['19', 'N-linked (GlcNAc...) asparagine'], ['272', 'N-linked (GlcNAc...) asparagine']], 'Q9UPC5': [['28', 'N-linked (GlcNAc...) asparagine'], ['36', 'N-linked (GlcNAc...) asparagine'], ['42', 'N-linked (GlcNAc...) asparagine'], ['200', 'N-linked (GlcNAc...) asparagine'], ['295', 'N-linked (GlcNAc...) asparagine']], 'Q9HC97': [['2', 'N-linked (GlcNAc...) asparagine']], 'O15354': [['36', 'N-linked (GlcNAc...) asparagine'], ['222', 'N-linked (GlcNAc...) asparagine'], ['239', 'N-linked (GlcNAc...) asparagine']], 'O60883': [['79', 'O-linked (GalNAc...) threonine'], ['85', 'O-linked (GalNAc...) threonine'], ['86', 'O-linked (GalNAc...) serine'], ['95', 'O-linked (GalNAc...) threonine'], ['105', 'N-linked (GlcNAc...) asparagine'], ['107', 'O-linked (GalNAc...) threonine']], 'O43194': [['192', 'N-linked (GlcNAc...) asparagine'], ['206', 'N-linked (GlcNAc...) asparagine'], ['212', 'N-linked (GlcNAc...) asparagine']], 'Q9Y5Y3': [['4', 'N-linked (GlcNAc...) asparagine'], ['17', 'N-linked (GlcNAc...) asparagine'], ['20', 'N-linked (GlcNAc...) asparagine']], 'Q13585': [], 'Q9Y2T5': [['2', 'N-linked (GlcNAc...) asparagine'], ['13', 'N-linked (GlcNAc...) asparagine'], ['20', 'N-linked (GlcNAc...) asparagine']], 'Q9BZJ8': [['12', 'N-linked (GlcNAc...) asparagine']], 'Q9BZJ7': [['3', 'N-linked (GlcNAc...) asparagine'], ['8', 'N-linked (GlcNAc...) asparagine']], 'Q9BZJ6': [['16', 'N-linked (GlcNAc...) asparagine'], ['28', 'N-linked (GlcNAc...) asparagine'], ['62', 'N-linked (GlcNAc...) asparagine']], 'Q8IYL9': [['2', 'N-linked (GlcNAc...) asparagine'], ['79', 'N-linked (GlcNAc...) asparagine'], ['166', 'N-linked (GlcNAc...) asparagine']], 'Q15743': [['3', 'N-linked (GlcNAc...) asparagine'], ['8', 'N-linked (GlcNAc...) asparagine']], 'O95800': [['2', 'N-linked (GlcNAc...) asparagine'], ['12', 'N-linked (GlcNAc...) asparagine'], ['25', 'N-linked (GlcNAc...) asparagine']], 'Q96P69': [], 'Q96P67': [['3', 'N-linked (GlcNAc...) asparagine'], ['4', 'N-linked (GlcNAc...) asparagine']], 'Q9NYM4': [['37', 'N-linked (GlcNAc...) asparagine'], ['46', 'N-linked (GlcNAc...) asparagine'], ['134', 'N-linked (GlcNAc...) asparagine']], 'Q9NQS5': [['3', 'N-linked (GlcNAc...) asparagine'], ['8', 'N-linked (GlcNAc...) asparagine']], 'P60893': [['3', 'N-linked (GlcNAc...) asparagine'], ['83', 'N-linked (GlcNAc...) asparagine'], ['182', 'N-linked (GlcNAc...) asparagine']], 'Q9BY21': [['4', 'N-linked (GlcNAc...) asparagine'], ['25', 'N-linked (GlcNAc...) asparagine'], ['33', 'N-linked (GlcNAc...) asparagine']], 'Q9GZN0': [['3', 'N-linked (GlcNAc...) asparagine']], 'Q96P66': [['7', 'N-linked (GlcNAc...) asparagine'], ['13', 'N-linked (GlcNAc...) asparagine']], 'Q9UNW8': [['35', 'N-linked (GlcNAc...) asparagine']], 'Q8IZ08': [], 'Q6DWJ6': [['11', 'N-linked (GlcNAc...) asparagine']], 'Q7Z602': [['5', 'N-linked (GlcNAc...) asparagine'], ['9', 'N-linked (GlcNAc...) asparagine']], 'Q7Z601': [], 'Q96CH1': [['8', 'N-linked (GlcNAc...) asparagine']], 'Q8TDV2': [['34', 'N-linked (GlcNAc...) asparagine']], 'Q86SP6': [['7', 'N-linked (GlcNAc...) asparagine'], ['11', 'N-linked (GlcNAc...) asparagine'], ['21', 'N-linked (GlcNAc...) asparagine']], 'Q8NGU9': [], 'Q8TDV0': [['10', 'N-linked (GlcNAc...) asparagine'], ['15', 'N-linked (GlcNAc...) asparagine']], 'Q8TDT2': [], 'Q6NV75': [], 'Q9UJ42': [['8', 'N-linked (GlcNAc...) asparagine']], 'Q8N6U8': [['4', 'N-linked (GlcNAc...) asparagine'], ['15', 'N-linked (GlcNAc...) asparagine']], 'Q16538': [['86', 'N-linked (GlcNAc...) asparagine']], 'O14626': [['3', 'N-linked (GlcNAc...) asparagine']], 'Q9NS66': [['3', 'N-linked (GlcNAc...) asparagine'], ['184', 'N-linked (GlcNAc...) asparagine']], 'Q9BXC1': [['4', 'N-linked (GlcNAc...) asparagine'], ['164', 'N-linked (GlcNAc...) asparagine']], 'Q14439': [['4', 'N-linked (GlcNAc...) asparagine'], ['11', 'N-linked (GlcNAc...) asparagine'], ['17', 'N-linked (GlcNAc...) asparagine'], ['27', 'N-linked (GlcNAc...) asparagine']], 'O15218': [['28', 'N-linked (GlcNAc...) asparagine'], ['37', 'N-linked (GlcNAc...) asparagine']], 'P32249': [], 'Q9BXB1': [['68', 'N-linked (GlcNAc...) asparagine'], ['199', 'N-linked (GlcNAc...) asparagine'], ['294', 'N-linked (GlcNAc...) asparagine'], ['314', 'N-linked (GlcNAc...) asparagine'], ['505', 'N-linked (GlcNAc...) asparagine']], 'O75473': [['63', 'N-linked (GlcNAc...) asparagine'], ['77', 'N-linked (GlcNAc...) asparagine'], ['208', 'N-linked (GlcNAc...) asparagine'], ['500', 'N-linked (GlcNAc...) asparagine'], ['792', 'N-linked (GlcNAc...) asparagine']], 'Q9HBX8': [['77', 'N-linked (GlcNAc...) asparagine'], ['208', 'N-linked (GlcNAc...) asparagine']], 'P04201': [['5', 'N-linked (GlcNAc...) asparagine'], ['16', 'N-linked (GlcNAc...) asparagine'], ['22', 'N-linked (GlcNAc...) asparagine']], 'P35410': [['54', 'N-linked (GlcNAc...) asparagine'], ['57', 'N-linked (GlcNAc...) asparagine']], 'Q8TDS7': [['2', 'N-linked (GlcNAc...) asparagine'], ['6', 'N-linked (GlcNAc...) asparagine'], ['16', 'N-linked (GlcNAc...) asparagine'], ['92', 'N-linked (GlcNAc...) asparagine']], 'Q86SM8': [], 'Q96AM1': [['4', 'N-linked (GlcNAc...) asparagine']], 'Q86SM5': [], 'Q96LB2': [['16', 'N-linked (GlcNAc...) asparagine']], 'Q96LB1': [], 'Q96LB0': [], 'Q96LA9': [['25', 'N-linked (GlcNAc...) asparagine'], ['89', 'N-linked (GlcNAc...) asparagine']], 'Q86VZ1': [['5', 'N-linked (GlcNAc...) asparagine'], ['11', 'N-linked (GlcNAc...) asparagine']], 'O00398': [['16', 'N-linked (GlcNAc...) asparagine'], ['26', 'N-linked (GlcNAc...) asparagine'], ['178', 'N-linked (GlcNAc...) asparagine']], 'Q9P1P5': [['24', 'N-linked (GlcNAc...) asparagine'], ['30', 'N-linked (GlcNAc...) asparagine'], ['289', 'N-linked (GlcNAc...) asparagine']], 'O14804': [['21', 'N-linked (GlcNAc...) asparagine']], 'Q96RI8': [['4', 'N-linked (GlcNAc...) asparagine'], ['19', 'N-linked (GlcNAc...) asparagine']], 'Q969N4': [['4', 'N-linked (GlcNAc...) asparagine'], ['18', 'N-linked (GlcNAc...) asparagine']], 'Q96RI9': [['4', 'N-linked (GlcNAc...) asparagine'], ['19', 'N-linked (GlcNAc...) asparagine']], 'Q16602': [['66', 'N-linked (GlcNAc...) asparagine'], ['118', 'N-linked (GlcNAc...) asparagine'], ['123', 'N-linked (GlcNAc...) asparagine']], 'Q13324': [['13', 'N-linked (GlcNAc...) asparagine'], ['41', 'N-linked (GlcNAc...) asparagine'], ['74', 'N-linked (GlcNAc...) asparagine'], ['86', 'N-linked (GlcNAc...) asparagine'], ['94', 'N-linked (GlcNAc...) asparagine']], 'Q02643': [['50', 'N-linked (GlcNAc...) asparagine']], 'P48546': [['62', 'N-linked (GlcNAc...) asparagine'], ['77', 'N-linked (GlcNAc...) asparagine']], 'P43220': [['63', 'N-linked (GlcNAc...) asparagine'], ['82', 'N-linked (GlcNAc...) asparagine'], ['115', 'N-linked (GlcNAc...) asparagine']], 'O95838': [['97', 'N-linked (GlcNAc...) asparagine'], ['113', 'N-linked (GlcNAc...) asparagine'], ['148', 'N-linked (GlcNAc...) asparagine'], ['162', 'N-linked (GlcNAc...) asparagine']], 'P47871': [['46', 'N-linked (GlcNAc...) asparagine'], ['59', 'N-linked (GlcNAc...) asparagine'], ['74', 'N-linked (GlcNAc...) asparagine'], ['78', 'N-linked (GlcNAc...) asparagine']], 'P47872': [['72', 'N-linked (GlcNAc...) asparagine'], ['100', 'N-linked (GlcNAc...) asparagine'], ['106', 'N-linked (GlcNAc...) asparagine'], ['128', 'N-linked (GlcNAc...) asparagine'], ['291', 'N-linked (GlcNAc...) asparagine']], 'Q03431': [['151', 'N-linked (GlcNAc...) asparagine'], ['161', 'N-linked (GlcNAc...) asparagine'], ['166', 'N-linked (GlcNAc...) asparagine'], ['176', 'N-linked (GlcNAc...) asparagine']], 'P49190': [['51', 'N-linked (GlcNAc...) asparagine'], ['106', 'N-linked (GlcNAc...) asparagine'], ['116', 'N-linked (GlcNAc...) asparagine'], ['121', 'N-linked (GlcNAc...) asparagine']], 'P41586': [['48', 'N-linked (GlcNAc...) asparagine'], ['60', 'N-linked (GlcNAc...) asparagine'], ['117', 'N-linked (GlcNAc...) asparagine'], ['300', 'N-linked (GlcNAc...) asparagine'], ['375', 'N-linked (GlcNAc...) asparagine']], 'P32241': [['58', 'N-linked (GlcNAc...) asparagine'], ['69', 'N-linked (GlcNAc...) asparagine'], ['100', 'N-linked (GlcNAc...) asparagine'], ['290', 'N-linked (GlcNAc...) asparagine']], 'P41587': [['58', 'N-linked (GlcNAc...) asparagine'], ['88', 'N-linked (GlcNAc...) asparagine'], ['92', 'N-linked (GlcNAc...) asparagine']], 'O94910': [['98', 'N-linked (GlcNAc...) asparagine'], ['531', 'N-linked (GlcNAc...) asparagine'], ['640', 'N-linked (GlcNAc...) asparagine'], ['742', 'N-linked (GlcNAc...) asparagine'], ['801', 'N-linked (GlcNAc...) asparagine'], ['806', 'N-linked (GlcNAc...) asparagine'], ['827', 'N-linked (GlcNAc...) asparagine']], 'O95490': [['99', 'N-linked (GlcNAc...) asparagine'], ['331', 'N-linked (GlcNAc...) asparagine'], ['520', 'N-linked (GlcNAc...) asparagine'], ['629', 'N-linked (GlcNAc...) asparagine'], ['731', 'N-linked (GlcNAc...) asparagine'], ['744', 'N-linked (GlcNAc...) asparagine'], ['787', 'N-linked (GlcNAc...) asparagine'], ['792', 'N-linked (GlcNAc...) asparagine'], ['813', 'N-linked (GlcNAc...) asparagine']], 'Q9HAR2': [['93', 'N-linked (GlcNAc...) asparagine'], ['464', 'N-linked (GlcNAc...) asparagine'], ['549', 'N-linked (GlcNAc...) asparagine'], ['746', 'N-linked (GlcNAc...) asparagine'], ['759', 'N-linked (GlcNAc...) asparagine'], ['804', 'N-linked (GlcNAc...) asparagine'], ['830', 'N-linked (GlcNAc...) asparagine'], ['1076', 'N-linked (GlcNAc...) asparagine']], 'Q9HBW9': [['21', 'N-linked (GlcNAc...) asparagine'], ['64', 'N-linked (GlcNAc...) asparagine'], ['74', 'N-linked (GlcNAc...) asparagine'], ['127', 'N-linked (GlcNAc...) asparagine'], ['177', 'N-linked (GlcNAc...) asparagine'], ['188', 'N-linked (GlcNAc...) asparagine'], ['249', 'N-linked (GlcNAc...) asparagine'], ['381', 'N-linked (GlcNAc...) asparagine'], ['395', 'N-linked (GlcNAc...) asparagine']], 'Q14246': [['94', 'N-linked (GlcNAc...) asparagine'], ['99', 'N-linked (GlcNAc...) asparagine'], ['127', 'N-linked (GlcNAc...) asparagine'], ['167', 'N-linked (GlcNAc...) asparagine'], ['189', 'N-linked (GlcNAc...) asparagine'], ['194', 'N-linked (GlcNAc...) asparagine'], ['232', 'N-linked (GlcNAc...) asparagine'], ['258', 'N-linked (GlcNAc...) asparagine'], ['312', 'N-linked (GlcNAc...) asparagine'], ['366', 'N-linked (GlcNAc...) asparagine'], ['375', 'N-linked (GlcNAc...) asparagine'], ['448', 'N-linked (GlcNAc...) asparagine']], 'Q9UHX3': [['41', 'N-linked (GlcNAc...) asparagine'], ['111', 'N-linked (GlcNAc...) asparagine'], ['206', 'N-linked (GlcNAc...) asparagine'], ['298', 'N-linked (GlcNAc...) asparagine'], ['347', 'N-linked (GlcNAc...) asparagine'], ['354', 'N-linked (GlcNAc...) asparagine'], ['456', 'N-linked (GlcNAc...) asparagine'], ['460', 'N-linked (GlcNAc...) asparagine']], 'Q9BY15': [['34', 'N-linked (GlcNAc...) asparagine'], ['39', 'N-linked (GlcNAc...) asparagine'], ['145', 'N-linked (GlcNAc...) asparagine'], ['189', 'N-linked (GlcNAc...) asparagine'], ['202', 'N-linked (GlcNAc...) asparagine'], ['250', 'N-linked (GlcNAc...) asparagine'], ['279', 'N-linked (GlcNAc...) asparagine'], ['327', 'N-linked (GlcNAc...) asparagine'], ['334', 'N-linked (GlcNAc...) asparagine']], 'P48960': [['33', 'N-linked (GlcNAc...) asparagine'], ['38', 'N-linked (GlcNAc...) asparagine'], ['108', 'N-linked (GlcNAc...) asparagine'], ['203', 'N-linked (GlcNAc...) asparagine'], ['371', 'N-linked (GlcNAc...) asparagine'], ['406', 'N-linked (GlcNAc...) asparagine'], ['413', 'N-linked (GlcNAc...) asparagine'], ['453', 'N-linked (GlcNAc...) asparagine'], ['520', 'N-linked (GlcNAc...) asparagine']], 'Q86SQ6': [['75', 'N-linked (GlcNAc...) asparagine']], 'Q96PE1': [['84', 'N-linked (GlcNAc...) asparagine'], ['101', 'N-linked (GlcNAc...) asparagine'], ['162', 'N-linked (GlcNAc...) asparagine'], ['207', 'N-linked (GlcNAc...) asparagine'], ['275', 'N-linked (GlcNAc...) asparagine'], ['602', 'N-linked (GlcNAc...) asparagine'], ['690', 'N-linked (GlcNAc...) asparagine']], 'Q8IWK6': [['81', 'N-linked (GlcNAc...) asparagine'], ['98', 'N-linked (GlcNAc...) asparagine'], ['159', 'N-linked (GlcNAc...) asparagine'], ['206', 'N-linked (GlcNAc...) asparagine'], ['301', 'N-linked (GlcNAc...) asparagine'], ['332', 'N-linked (GlcNAc...) asparagine'], ['433', 'N-linked (GlcNAc...) asparagine'], ['453', 'N-linked (GlcNAc...) asparagine'], ['592', 'N-linked (GlcNAc...) asparagine'], ['652', 'N-linked (GlcNAc...) asparagine'], ['687', 'N-linked (GlcNAc...) asparagine'], ['728', 'N-linked (GlcNAc...) asparagine'], ['821', 'N-linked (GlcNAc...) asparagine']], 'Q9NYQ6': [['403', 'N-linked (GlcNAc...) asparagine'], ['546', 'N-linked (GlcNAc...) asparagine'], ['634', 'N-linked (GlcNAc...) asparagine'], ['778', 'N-linked (GlcNAc...) asparagine'], ['1114', 'N-linked (GlcNAc...) asparagine'], ['1139', 'N-linked (GlcNAc...) asparagine'], ['1213', 'N-linked (GlcNAc...) asparagine'], ['1249', 'N-linked (GlcNAc...) asparagine'], ['1259', 'N-linked (GlcNAc...) asparagine'], ['1287', 'N-linked (GlcNAc...) asparagine'], ['1576', 'N-linked (GlcNAc...) asparagine'], ['1623', 'N-linked (GlcNAc...) asparagine'], ['1640', 'N-linked (GlcNAc...) asparagine'], ['1979', 'N-linked (GlcNAc...) asparagine'], ['2103', 'N-linked (GlcNAc...) asparagine'], ['2122', 'N-linked (GlcNAc...) asparagine'], ['2257', 'N-linked (GlcNAc...) asparagine'], ['2415', 'N-linked (GlcNAc...) asparagine'], ['2437', 'N-linked (GlcNAc...) asparagine'], ['2523', 'N-linked (GlcNAc...) asparagine']], 'Q9HCU4': [['486', 'N-linked (GlcNAc...) asparagine'], ['557', 'N-linked (GlcNAc...) asparagine'], ['701', 'N-linked (GlcNAc...) asparagine'], ['1036', 'N-linked (GlcNAc...) asparagine'], ['1076', 'N-linked (GlcNAc...) asparagine'], ['1182', 'N-linked (GlcNAc...) asparagine'], ['1212', 'N-linked (GlcNAc...) asparagine'], ['1501', 'N-linked (GlcNAc...) asparagine'], ['1565', 'N-linked (GlcNAc...) asparagine'], ['1741', 'N-linked (GlcNAc...) asparagine'], ['1827', 'N-linked (GlcNAc...) asparagine'], ['1900', 'N-linked (GlcNAc...) asparagine'], ['2024', 'N-linked (GlcNAc...) asparagine'], ['2043', 'N-linked (GlcNAc...) asparagine'], ['2061', 'N-linked (GlcNAc...) asparagine'], ['2323', 'N-linked (GlcNAc...) asparagine'], ['2345', 'N-linked (GlcNAc...) asparagine']], 'Q9NYQ7': [['632', 'N-linked (GlcNAc...) asparagine'], ['847', 'N-linked (GlcNAc...) asparagine'], ['1182', 'N-linked (GlcNAc...) asparagine'], ['1222', 'N-linked (GlcNAc...) asparagine'], ['1317', 'N-linked (GlcNAc...) asparagine'], ['1327', 'N-linked (GlcNAc...) asparagine'], ['1649', 'N-linked (GlcNAc...) asparagine'], ['1713', 'N-linked (GlcNAc...) asparagine'], ['1770', 'N-linked (GlcNAc...) asparagine'], ['2053', 'N-linked (GlcNAc...) asparagine'], ['2177', 'N-linked (GlcNAc...) asparagine'], ['2196', 'N-linked (GlcNAc...) asparagine'], ['2386', 'N-linked (GlcNAc...) asparagine'], ['2474', 'N-linked (GlcNAc...) asparagine'], ['2506', 'N-linked (GlcNAc...) asparagine']], 'Q6QNK2': [['90', 'N-linked (GlcNAc...) asparagine'], ['185', 'N-linked (GlcNAc...) asparagine'], ['282', 'N-linked (GlcNAc...) asparagine'], ['302', 'N-linked (GlcNAc...) asparagine'], ['319', 'N-linked (GlcNAc...) asparagine'], ['394', 'N-linked (GlcNAc...) asparagine'], ['476', 'N-linked (GlcNAc...) asparagine'], ['501', 'N-linked (GlcNAc...) asparagine'], ['533', 'N-linked (GlcNAc...) asparagine']], 'Q7Z7M1': [['271', 'N-linked (GlcNAc...) asparagine'], ['634', 'N-linked (GlcNAc...) asparagine']], 'Q5T601': [['139', 'N-linked (GlcNAc...) asparagine'], ['168', 'N-linked (GlcNAc...) asparagine'], ['205', 'N-linked (GlcNAc...) asparagine'], ['282', 'N-linked (GlcNAc...) asparagine'], ['310', 'N-linked (GlcNAc...) asparagine'], ['317', 'N-linked (GlcNAc...) asparagine'], ['329', 'N-linked (GlcNAc...) asparagine'], ['354', 'N-linked (GlcNAc...) asparagine'], ['368', 'N-linked (GlcNAc...) asparagine'], ['389', 'N-linked (GlcNAc...) asparagine'], ['410', 'N-linked (GlcNAc...) asparagine'], ['423', 'N-linked (GlcNAc...) asparagine'], ['437', 'N-linked (GlcNAc...) asparagine'], ['455', 'N-linked (GlcNAc...) asparagine'], ['512', 'N-linked (GlcNAc...) asparagine'], ['528', 'N-linked (GlcNAc...) asparagine'], ['553', 'N-linked (GlcNAc...) asparagine'], ['736', 'N-linked (GlcNAc...) asparagine'], ['739', 'N-linked (GlcNAc...) asparagine']], 'Q8IZF7': [['21', 'N-linked (GlcNAc...) asparagine'], ['220', 'N-linked (GlcNAc...) asparagine'], ['252', 'N-linked (GlcNAc...) asparagine'], ['260', 'N-linked (GlcNAc...) asparagine'], ['305', 'N-linked (GlcNAc...) asparagine'], ['313', 'N-linked (GlcNAc...) asparagine'], ['358', 'N-linked (GlcNAc...) asparagine']], 'Q8IZF5': [['188', 'N-linked (GlcNAc...) asparagine'], ['264', 'N-linked (GlcNAc...) asparagine'], ['301', 'N-linked (GlcNAc...) asparagine'], ['382', 'N-linked (GlcNAc...) asparagine'], ['441', 'N-linked (GlcNAc...) asparagine'], ['648', 'N-linked (GlcNAc...) asparagine']], 'Q8IZF3': [['61', 'N-linked (GlcNAc...) asparagine'], ['169', 'N-linked (GlcNAc...) asparagine'], ['177', 'N-linked (GlcNAc...) asparagine'], ['209', 'N-linked (GlcNAc...) asparagine'], ['229', 'N-linked (GlcNAc...) asparagine'], ['250', 'N-linked (GlcNAc...) asparagine'], ['257', 'N-linked (GlcNAc...) asparagine'], ['263', 'N-linked (GlcNAc...) asparagine'], ['264', 'N-linked (GlcNAc...) asparagine'], ['286', 'N-linked (GlcNAc...) asparagine'], ['309', 'N-linked (GlcNAc...) asparagine'], ['340', 'N-linked (GlcNAc...) asparagine'], ['379', 'N-linked (GlcNAc...) asparagine']], 'Q8IZF2': [['73', 'N-linked (GlcNAc...) asparagine'], ['94', 'N-linked (GlcNAc...) asparagine'], ['106', 'N-linked (GlcNAc...) asparagine'], ['188', 'N-linked (GlcNAc...) asparagine'], ['256', 'N-linked (GlcNAc...) asparagine'], ['272', 'N-linked (GlcNAc...) asparagine'], ['301', 'N-linked (GlcNAc...) asparagine'], ['315', 'N-linked (GlcNAc...) asparagine'], ['328', 'N-linked (GlcNAc...) asparagine'], ['398', 'N-linked (GlcNAc...) asparagine'], ['472', 'N-linked (GlcNAc...) asparagine'], ['487', 'N-linked (GlcNAc...) asparagine'], ['505', 'N-linked (GlcNAc...) asparagine'], ['540', 'N-linked (GlcNAc...) asparagine'], ['627', 'N-linked (GlcNAc...) asparagine'], ['649', 'N-linked (GlcNAc...) asparagine'], ['666', 'N-linked (GlcNAc...) asparagine'], ['820', 'N-linked (GlcNAc...) asparagine'], ['931', 'N-linked (GlcNAc...) asparagine'], ['963', 'N-linked (GlcNAc...) asparagine'], ['982', 'N-linked (GlcNAc...) asparagine']], 'O14514': [['64', 'N-linked (GlcNAc...) asparagine'], ['401', 'N-linked (GlcNAc...) asparagine'], ['607', 'N-linked (GlcNAc...) asparagine'], ['692', 'N-linked (GlcNAc...) asparagine'], ['844', 'N-linked (GlcNAc...) asparagine'], ['877', 'N-linked (GlcNAc...) asparagine'], ['881', 'N-linked (GlcNAc...) asparagine']], 'O60241': [['106', 'N-linked (GlcNAc...) asparagine'], ['191', 'N-linked (GlcNAc...) asparagine'], ['192', 'N-linked (GlcNAc...) asparagine'], ['356', 'N-linked (GlcNAc...) asparagine'], ['437', 'N-linked (GlcNAc...) asparagine'], ['560', 'N-linked (GlcNAc...) asparagine'], ['645', 'N-linked (GlcNAc...) asparagine'], ['867', 'N-linked (GlcNAc...) asparagine']], 'O60242': [['51', 'N-linked (GlcNAc...) asparagine'], ['54', 'N-linked (GlcNAc...) asparagine'], ['82', 'N-linked (GlcNAc...) asparagine'], ['105', 'N-linked (GlcNAc...) asparagine'], ['241', 'N-linked (GlcNAc...) asparagine'], ['337', 'N-linked (GlcNAc...) asparagine'], ['418', 'N-linked (GlcNAc...) asparagine'], ['540', 'N-linked (GlcNAc...) asparagine'], ['625', 'N-linked (GlcNAc...) asparagine'], ['779', 'N-linked (GlcNAc...) asparagine'], ['812', 'N-linked (GlcNAc...) asparagine'], ['828', 'N-linked (GlcNAc...) asparagine'], ['937', 'N-linked (GlcNAc...) asparagine']], 'Q9Y653': [['39', 'N-linked (GlcNAc...) asparagine'], ['148', 'N-linked (GlcNAc...) asparagine'], ['171', 'N-linked (GlcNAc...) asparagine'], ['234', 'N-linked (GlcNAc...) asparagine'], ['303', 'N-linked (GlcNAc...) asparagine'], ['324', 'N-linked (GlcNAc...) asparagine'], ['341', 'N-linked (GlcNAc...) asparagine']], 'Q8IZP9': [['44', 'N-linked (GlcNAc...) asparagine'], ['85', 'N-linked (GlcNAc...) asparagine'], ['99', 'N-linked (GlcNAc...) asparagine'], ['111', 'N-linked (GlcNAc...) asparagine'], ['117', 'N-linked (GlcNAc...) asparagine'], ['144', 'N-linked (GlcNAc...) asparagine'], ['162', 'N-linked (GlcNAc...) asparagine'], ['186', 'N-linked (GlcNAc...) asparagine'], ['194', 'N-linked (GlcNAc...) asparagine'], ['357', 'N-linked (GlcNAc...) asparagine'], ['370', 'N-linked (GlcNAc...) asparagine'], ['435', 'N-linked (GlcNAc...) asparagine'], ['438', 'N-linked (GlcNAc...) asparagine'], ['456', 'N-linked (GlcNAc...) asparagine'], ['461', 'N-linked (GlcNAc...) asparagine'], ['528', 'N-linked (GlcNAc...) asparagine'], ['542', 'N-linked (GlcNAc...) asparagine'], ['547', 'N-linked (GlcNAc...) asparagine'], ['551', 'N-linked (GlcNAc...) asparagine'], ['597', 'N-linked (GlcNAc...) asparagine'], ['857', 'N-linked (GlcNAc...) asparagine']], 'Q86Y34': [['98', 'N-linked (GlcNAc...) asparagine'], ['144', 'N-linked (GlcNAc...) asparagine'], ['210', 'N-linked (GlcNAc...) asparagine'], ['413', 'N-linked (GlcNAc...) asparagine']], 'Q8IZF6': [['233', 'N-linked (GlcNAc...) asparagine'], ['487', 'N-linked (GlcNAc...) asparagine'], ['836', 'N-linked (GlcNAc...) asparagine'], ['899', 'N-linked (GlcNAc...) asparagine'], ['1020', 'N-linked (GlcNAc...) asparagine'], ['1519', 'N-linked (GlcNAc...) asparagine'], ['2361', 'N-linked (GlcNAc...) asparagine'], ['2640', 'N-linked (GlcNAc...) asparagine']], 'Q8IZF4': [['58', 'N-linked (GlcNAc...) asparagine'], ['65', 'N-linked (GlcNAc...) asparagine'], ['146', 'N-linked (GlcNAc...) asparagine'], ['147', 'N-linked (GlcNAc...) asparagine'], ['173', 'N-linked (GlcNAc...) asparagine'], ['179', 'N-linked (GlcNAc...) asparagine'], ['394', 'N-linked (GlcNAc...) asparagine'], ['400', 'N-linked (GlcNAc...) asparagine']], 'Q86SQ4': [['121', 'N-linked (GlcNAc...) asparagine'], ['143', 'N-linked (GlcNAc...) asparagine'], ['206', 'N-linked (GlcNAc...) asparagine'], ['258', 'N-linked (GlcNAc...) asparagine'], ['314', 'N-linked (GlcNAc...) asparagine'], ['324', 'N-linked (GlcNAc...) asparagine'], ['353', 'N-linked (GlcNAc...) asparagine'], ['438', 'N-linked (GlcNAc...) asparagine'], ['445', 'N-linked (GlcNAc...) asparagine'], ['452', 'N-linked (GlcNAc...) asparagine'], ['485', 'N-linked (GlcNAc...) asparagine'], ['488', 'N-linked (GlcNAc...) asparagine'], ['505', 'N-linked (GlcNAc...) asparagine'], ['563', 'N-linked (GlcNAc...) asparagine'], ['593', 'N-linked (GlcNAc...) asparagine'], ['600', 'N-linked (GlcNAc...) asparagine'], ['605', 'N-linked (GlcNAc...) asparagine'], ['667', 'N-linked (GlcNAc...) asparagine'], ['673', 'N-linked (GlcNAc...) asparagine'], ['695', 'N-linked (GlcNAc...) asparagine'], ['704', 'N-linked (GlcNAc...) asparagine'], ['750', 'N-linked (GlcNAc...) asparagine'], ['776', 'N-linked (GlcNAc...) asparagine'], ['811', 'N-linked (GlcNAc...) asparagine'], ['818', 'N-linked (GlcNAc...) asparagine']], 'Q96K78': [['82', 'N-linked (GlcNAc...) asparagine'], ['159', 'N-linked (GlcNAc...) asparagine'], ['178', 'N-linked (GlcNAc...) asparagine'], ['191', 'N-linked (GlcNAc...) asparagine'], ['247', 'N-linked (GlcNAc...) asparagine'], ['261', 'N-linked (GlcNAc...) asparagine'], ['312', 'N-linked (GlcNAc...) asparagine'], ['316', 'N-linked (GlcNAc...) asparagine'], ['387', 'N-linked (GlcNAc...) asparagine'], ['413', 'N-linked (GlcNAc...) asparagine']], 'Q8WXG9': [], 'P41180': [['90', 'N-linked (GlcNAc...) asparagine'], ['130', 'N-linked (GlcNAc...) asparagine'], ['261', 'N-linked (GlcNAc...) asparagine'], ['287', 'N-linked (GlcNAc...) asparagine'], ['386', 'N-linked (GlcNAc...) asparagine'], ['400', 'N-linked (GlcNAc...) asparagine'], ['446', 'N-linked (GlcNAc...) asparagine'], ['468', 'N-linked (GlcNAc...) asparagine'], ['488', 'N-linked (GlcNAc...) asparagine'], ['541', 'N-linked (GlcNAc...) asparagine'], ['594', 'N-linked (GlcNAc...) asparagine']], 'Q9UBS5': [['24', 'N-linked (GlcNAc...) asparagine'], ['84', 'N-linked (GlcNAc...) asparagine'], ['409', 'N-linked (GlcNAc...) asparagine'], ['440', 'N-linked (GlcNAc...) asparagine'], ['482', 'N-linked (GlcNAc...) asparagine'], ['502', 'N-linked (GlcNAc...) asparagine'], ['514', 'N-linked (GlcNAc...) asparagine']], 'O75899': [['90', 'N-linked (GlcNAc...) asparagine'], ['298', 'N-linked (GlcNAc...) asparagine'], ['389', 'N-linked (GlcNAc...) asparagine'], ['404', 'N-linked (GlcNAc...) asparagine'], ['453', 'N-linked (GlcNAc...) asparagine']], 'Q13255': [['98', 'N-linked (GlcNAc...) asparagine'], ['223', 'N-linked (GlcNAc...) asparagine'], ['397', 'N-linked (GlcNAc...) asparagine'], ['515', 'N-linked (GlcNAc...) asparagine']], 'Q14416': [['203', 'N-linked (GlcNAc...) asparagine'], ['286', 'N-linked (GlcNAc...) asparagine'], ['338', 'N-linked (GlcNAc...) asparagine'], ['402', 'N-linked (GlcNAc...) asparagine'], ['547', 'N-linked (GlcNAc...) asparagine']], 'Q14832': [['209', 'N-linked (GlcNAc...) asparagine'], ['292', 'N-linked (GlcNAc...) asparagine'], ['414', 'N-linked (GlcNAc...) asparagine'], ['439', 'N-linked (GlcNAc...) asparagine']], 'Q14833': [['98', 'N-linked (GlcNAc...) asparagine'], ['301', 'N-linked (GlcNAc...) asparagine'], ['454', 'N-linked (GlcNAc...) asparagine'], ['484', 'N-linked (GlcNAc...) asparagine'], ['569', 'N-linked (GlcNAc...) asparagine']], 'P41594': [['88', 'N-linked (GlcNAc...) asparagine'], ['210', 'N-linked (GlcNAc...) asparagine'], ['378', 'N-linked (GlcNAc...) asparagine'], ['382', 'N-linked (GlcNAc...) asparagine'], ['445', 'N-linked (GlcNAc...) asparagine'], ['734', 'N-linked (GlcNAc...) asparagine']], 'O15303': [['296', 'N-linked (GlcNAc...) asparagine'], ['451', 'N-linked (GlcNAc...) asparagine'], ['479', 'N-linked (GlcNAc...) asparagine'], ['567', 'N-linked (GlcNAc...) asparagine']], 'Q14831': [['98', 'N-linked (GlcNAc...) asparagine'], ['458', 'N-linked (GlcNAc...) asparagine'], ['486', 'N-linked (GlcNAc...) asparagine'], ['572', 'N-linked (GlcNAc...) asparagine']], 'O00222': [['95', 'N-linked (GlcNAc...) asparagine'], ['298', 'N-linked (GlcNAc...) asparagine'], ['452', 'N-linked (GlcNAc...) asparagine'], ['480', 'N-linked (GlcNAc...) asparagine'], ['565', 'N-linked (GlcNAc...) asparagine']], 'Q7RTX1': [['87', 'N-linked (GlcNAc...) asparagine'], ['88', 'N-linked (GlcNAc...) asparagine'], ['95', 'N-linked (GlcNAc...) asparagine'], ['291', 'N-linked (GlcNAc...) asparagine'], ['479', 'N-linked (GlcNAc...) asparagine'], ['529', 'N-linked (GlcNAc...) asparagine']], 'Q8TE23': [['84', 'N-linked (GlcNAc...) asparagine'], ['248', 'N-linked (GlcNAc...) asparagine'], ['292', 'N-linked (GlcNAc...) asparagine'], ['312', 'N-linked (GlcNAc...) asparagine'], ['368', 'N-linked (GlcNAc...) asparagine'], ['428', 'N-linked (GlcNAc...) asparagine'], ['487', 'N-linked (GlcNAc...) asparagine'], ['527', 'N-linked (GlcNAc...) asparagine']], 'Q7RTX0': [['85', 'N-linked (GlcNAc...) asparagine'], ['130', 'N-linked (GlcNAc...) asparagine'], ['264', 'N-linked (GlcNAc...) asparagine'], ['285', 'N-linked (GlcNAc...) asparagine'], ['380', 'N-linked (GlcNAc...) asparagine'], ['411', 'N-linked (GlcNAc...) asparagine'], ['432', 'N-linked (GlcNAc...) asparagine'], ['475', 'N-linked (GlcNAc...) asparagine']], 'Q8NFN8': [['6', 'N-linked (GlcNAc...) asparagine']], 'Q5T848': [['98', 'N-linked (GlcNAc...) asparagine'], ['143', 'N-linked (GlcNAc...) asparagine'], ['215', 'N-linked (GlcNAc...) asparagine'], ['274', 'N-linked (GlcNAc...) asparagine'], ['333', 'N-linked (GlcNAc...) asparagine']], 'Q6PRD1': [['75', 'N-linked (GlcNAc...) asparagine'], ['298', 'N-linked (GlcNAc...) asparagine']], 'Q8NFJ5': [['158', 'N-linked (GlcNAc...) asparagine']], 'Q9NZH0': [['30', 'N-linked (GlcNAc...) asparagine']], 'Q9NQ84': [['191', 'N-linked (GlcNAc...) asparagine']], 'Q9NZD1': [], 'Q5T6X5': [['121', 'N-linked (GlcNAc...) asparagine'], ['259', 'N-linked (GlcNAc...) asparagine'], ['332', 'N-linked (GlcNAc...) asparagine'], ['378', 'N-linked (GlcNAc...) asparagine'], ['452', 'N-linked (GlcNAc...) asparagine'], ['555', 'N-linked (GlcNAc...) asparagine'], ['567', 'N-linked (GlcNAc...) asparagine'], ['590', 'N-linked (GlcNAc...) asparagine'], ['733', 'N-linked (GlcNAc...) asparagine']], 'Q9UP38': [['130', 'N-linked (GlcNAc...) asparagine'], ['231', 'N-linked (GlcNAc...) asparagine']], 'Q14332': [['53', 'N-linked (GlcNAc...) asparagine'], ['154', 'N-linked (GlcNAc...) asparagine']], 'Q9NPG1': [['42', 'N-linked (GlcNAc...) asparagine'], ['265', 'N-linked (GlcNAc...) asparagine'], ['356', 'N-linked (GlcNAc...) asparagine']], 'Q9ULV1': [['59', 'N-linked (GlcNAc...) asparagine'], ['144', 'N-linked (GlcNAc...) asparagine']], 'Q13467': [['47', 'N-linked (GlcNAc...) asparagine'], ['151', 'N-linked (GlcNAc...) asparagine']], 'O60353': [['38', 'N-linked (GlcNAc...) asparagine'], ['352', 'N-linked (GlcNAc...) asparagine']], 'O75084': [['63', 'N-linked (GlcNAc...) asparagine'], ['164', 'N-linked (GlcNAc...) asparagine']], 'Q9H461': [['49', 'N-linked (GlcNAc...) asparagine'], ['152', 'N-linked (GlcNAc...) asparagine'], ['475', 'N-linked (GlcNAc...) asparagine']], 'O00144': [['53', 'N-linked (GlcNAc...) asparagine'], ['158', 'N-linked (GlcNAc...) asparagine']], 'Q9ULW2': [['48', 'N-linked (GlcNAc...) asparagine'], ['153', 'N-linked (GlcNAc...) asparagine'], ['485', 'N-linked (GlcNAc...) asparagine']], 'Q9NYW7': [['163', 'N-linked (GlcNAc...) asparagine']], 'Q9NYW6': [['166', 'N-linked (GlcNAc...) asparagine']], 'Q9NYW5': [['164', 'N-linked (GlcNAc...) asparagine'], ['165', 'N-linked (GlcNAc...) asparagine'], ['169', 'N-linked (GlcNAc...) asparagine']], 'Q9NYW4': [['155', 'N-linked (GlcNAc...) asparagine']], 'Q9NYW3': [['167', 'N-linked (GlcNAc...) asparagine'], ['175', 'N-linked (GlcNAc...) asparagine']], 'Q9NYW2': [['167', 'N-linked (GlcNAc...) asparagine']], 'Q9NYW1': [['164', 'N-linked (GlcNAc...) asparagine']], 'Q9NYW0': [['92', 'N-linked (GlcNAc...) asparagine'], ['158', 'N-linked (GlcNAc...) asparagine']], 'Q9NYV9': [['162', 'N-linked (GlcNAc...) asparagine'], ['166', 'N-linked (GlcNAc...) asparagine']], 'Q9NYV8': [['153', 'N-linked (GlcNAc...) asparagine'], ['162', 'N-linked (GlcNAc...) asparagine'], ['171', 'N-linked (GlcNAc...) asparagine']], 'Q9NYV7': [['80', 'N-linked (GlcNAc...) asparagine'], ['163', 'N-linked (GlcNAc...) asparagine']], 'P59542': [['161', 'N-linked (GlcNAc...) asparagine']], 'P59543': [['161', 'N-linked (GlcNAc...) asparagine'], ['176', 'N-linked (GlcNAc...) asparagine']], 'P59541': [['161', 'N-linked (GlcNAc...) asparagine'], ['176', 'N-linked (GlcNAc...) asparagine']], 'P59538': [['161', 'N-linked (GlcNAc...) asparagine']], 'P59533': [['89', 'N-linked (GlcNAc...) asparagine'], ['178', 'N-linked (GlcNAc...) asparagine']], 'P59534': [['185', 'N-linked (GlcNAc...) asparagine'], ['194', 'N-linked (GlcNAc...) asparagine']], 'P59535': [['170', 'N-linked (GlcNAc...) asparagine'], ['179', 'N-linked (GlcNAc...) asparagine']], 'P59536': [['167', 'N-linked (GlcNAc...) asparagine']], 'Q7RTR8': [['163', 'N-linked (GlcNAc...) asparagine']], 'P59537': [['161', 'N-linked (GlcNAc...) asparagine'], ['176', 'N-linked (GlcNAc...) asparagine']], 'P59539': [['161', 'N-linked (GlcNAc...) asparagine']], 'P59540': [['161', 'N-linked (GlcNAc...) asparagine'], ['176', 'N-linked (GlcNAc...) asparagine']], 'P59544': [['161', 'N-linked (GlcNAc...) asparagine']], 'P59551': [['179', 'N-linked (GlcNAc...) asparagine']], 'Q5VW38': [['70', 'N-linked (GlcNAc...) asparagine'], ['169', 'N-linked (GlcNAc...) asparagine'], ['211', 'N-linked (GlcNAc...) asparagine']], 'Q96N19': [['4', 'N-linked (GlcNAc...) asparagine'], ['257', 'N-linked (GlcNAc...) asparagine']], 'Q8TCB6': [['7', 'N-linked (GlcNAc...) asparagine'], ['90', 'N-linked (GlcNAc...) asparagine']], 'Q86W33': [['11', 'N-linked (GlcNAc...) asparagine'], ['23', 'N-linked (GlcNAc...) asparagine'], ['361', 'N-linked (GlcNAc...) asparagine']], 'P51810': [['106', 'N-linked (GlcNAc...) asparagine']], 'Q5UAW9': []}


## Lipidation
Covalently attached lipid group(s)

In [6]:
# Build the dictionary keys as the entry names, values are lists [position, ptmtype]
dicPTMlipid=dict_ptm(os.path.join(resultspath,'remaining3/myprot_list_struc_remaining_merged_ENSG.csv'),'lipid')

P08100
Reponse from uniprot: Lipidation
LIPID 322;  /note="S-palmitoyl cysteine";  /evidence="ECO:0000250|UniProtKB:P02699"; LIPID 323;  /note="S-palmitoyl cysteine";  /evidence="ECO:0000250|UniProtKB:P02699"

res: [['322', 'S-palmitoyl cysteine'], ['323', 'S-palmitoyl cysteine']]
Q9UHM6
Reponse from uniprot: Lipidation


res: []
Q9H1Y3
Reponse from uniprot: Lipidation
LIPID 325;  /note="S-palmitoyl cysteine";  /evidence="ECO:0000250"

res: [['325', 'S-palmitoyl cysteine']]
Q6U736
Reponse from uniprot: Lipidation
LIPID 315;  /note="S-palmitoyl cysteine";  /evidence="ECO:0000255"; LIPID 316;  /note="S-palmitoyl cysteine";  /evidence="ECO:0000255"

res: [['315', 'S-palmitoyl cysteine'], ['316', 'S-palmitoyl cysteine']]
P03999
Reponse from uniprot: Lipidation


res: []
P04001
Reponse from uniprot: Lipidation


res: []
P04000
Reponse from uniprot: Lipidation


res: []
Q9P1Q5
Reponse from uniprot: Lipidation


res: []
P47890
Reponse from uniprot: Lipidation


res: []
Q8NH01
Reponse from uni

In [29]:
dicPTMlipid={'P18089': [['442', 'S-palmitoyl cysteine']], 'P21917': [['419', 'S-palmitoyl cysteine']], 'P35367': [], 'P48145': [], 'P48146': [], 'Q99463': [], 'Q9NSD7': [], 'P32745': [], 'Q15722': [], 'Q9NPC1': [], 'P0DMS8': [['303', 'S-palmitoyl cysteine']], 'P46091': [], 'Q99678': [], 'Q99680': [], 'Q96CH1': [], 'Q8TDV2': [], 'P35410': [], 'Q9P1P4': [], 'O00398': [], 'Q9P1P5': [], 'O14804': [], 'Q96RI8': [], 'Q969N4': [], 'Q96RI9': []}
# old {'P08908': [], 'P28222': [['388', 'S-palmitoyl cysteine']], 'P28221': [], 'P28566': [], 'P30939': [], 'P28223': [], 'P41595': [['397', 'S-palmitoyl cysteine']], 'P28335': [], 'Q13639': [['329', 'S-palmitoyl cysteine']], 'P47898': [], 'P50406': [], 'P34969': [['401', 'S-palmitoyl cysteine']], 'P11229': [], 'P08172': [], 'P20309': [], 'P08173': [], 'P08912': [], 'P35348': [['345', 'S-palmitoyl cysteine']], 'P35368': [['365', 'S-palmitoyl cysteine']], 'P25100': [['419', 'S-palmitoyl cysteine']], 'P08913': [['457', 'S-palmitoyl cysteine']], 'P18089': [['442', 'S-palmitoyl cysteine']], 'P18825': [], 'P08588': [['392', 'S-palmitoyl cysteine']], 'P07550': [['265', 'S-palmitoyl cysteine'], ['341', 'S-palmitoyl cysteine']], 'P13945': [['361', 'S-palmitoyl cysteine']], 'P21728': [['347', 'S-palmitoyl cysteine'], ['351', 'S-palmitoyl cysteine']], 'P14416': [['443', 'S-palmitoyl cysteine']], 'P35462': [], 'P21917': [['419', 'S-palmitoyl cysteine']], 'P21918': [['375', 'S-palmitoyl cysteine']], 'P35367': [], 'P25021': [['305', 'S-palmitoyl cysteine']], 'Q9Y5N1': [], 'Q9H3N8': [], 'Q96RJ0': [], 'P30556': [['355', 'S-palmitoyl cysteine']], 'P50052': [], 'P35414': [], 'P28336': [['341', 'S-palmitoyl cysteine']], 'P30550': [['339', 'S-palmitoyl cysteine']], 'P32247': [['347', 'S-palmitoyl cysteine']], 'P46663': [['330', 'S-palmitoyl cysteine']], 'P30411': [['351', 'S-palmitoyl cysteine']], 'P32238': [['387', 'S-palmitoyl cysteine']], 'P32239': [['408', 'S-palmitoyl cysteine']], 'Q16581': [], 'P21730': [], 'Q9P296': [], 'P25101': [], 'P24530': [['402', 'S-palmitoyl cysteine'], ['403', 'S-palmitoyl cysteine'], ['405', 'S-palmitoyl cysteine']], 'P21462': [], 'P25090': [], 'P25089': [], 'P47211': [['320', 'S-palmitoyl cysteine']], 'O43603': [], 'O60755': [['308', 'S-palmitoyl cysteine']], 'Q92847': [], 'P30968': [], 'Q969F8': [], 'Q99705': [], 'Q969V1': [], 'Q01726': [['315', 'S-palmitoyl cysteine']], 'Q01718': [['293', 'S-palmitoyl cysteine']], 'P41968': [['315', 'S-palmitoyl cysteine']], 'P32245': [['318', 'S-palmitoyl cysteine']], 'P33032': [['311', 'S-palmitoyl cysteine'], ['312', 'S-palmitoyl cysteine']], 'O43193': [], 'Q9HB89': [], 'Q9GZQ4': [], 'Q9GZQ6': [], 'Q9Y5X5': [], 'Q6W5P4': [], 'P48145': [], 'P48146': [], 'P25929': [['338', 'S-palmitoyl cysteine']], 'P49146': [['342', 'S-palmitoyl cysteine']], 'P50391': [['340', 'S-palmitoyl cysteine']], 'Q15761': [['442', 'S-palmitoyl cysteine']], 'Q99463': [], 'P30989': [['381', 'S-palmitoyl cysteine'], ['383', 'S-palmitoyl cysteine']], 'O95665': [['377', 'S-palmitoyl cysteine']], 'P41143': [['333', 'S-palmitoyl cysteine']], 'P41145': [['345', 'S-palmitoyl cysteine']], 'P35372': [['353', 'S-palmitoyl cysteine']], 'P41146': [['334', 'S-palmitoyl cysteine']], 'O43613': [], 'O43614': [], 'Q96P65': [], 'P49683': [], 'P25116': [], 'P55085': [['361', 'S-palmitoyl cysteine']], 'O00254': [], 'Q96RI0': [], 'Q9HBX9': [], 'Q8WXD0': [], 'Q9NSD7': [], 'Q8TDU9': [], 'P30872': [['339', 'S-palmitoyl cysteine']], 'P30874': [['328', 'S-palmitoyl cysteine']], 'P32745': [], 'P31391': [['327', 'S-palmitoyl cysteine']], 'P35346': [['320', 'S-palmitoyl cysteine']], 'P25103': [['322', 'S-palmitoyl cysteine']], 'P21452': [['324', 'S-palmitoyl cysteine']], 'P29371': [['374', 'S-palmitoyl cysteine']], 'P34981': [], 'Q9UKP6': [], 'P37288': [['365', 'S-palmitoyl cysteine'], ['366', 'S-palmitoyl cysteine']], 'P47901': [], 'P30518': [['341', 'S-palmitoyl cysteine'], ['342', 'S-palmitoyl cysteine']], 'P30559': [], 'Q99788': [], 'P32246': [], 'P41597': [], 'P51677': [], 'P51679': [], 'P51681': [['321', 'S-palmitoyl cysteine'], ['323', 'S-palmitoyl cysteine'], ['324', 'S-palmitoyl cysteine']], 'P51684': [], 'P32248': [], 'P51685': [], 'P51686': [], 'P46092': [], 'P25024': [], 'P25025': [], 'P49682': [], 'P61073': [], 'P32302': [], 'O00574': [], 'P49238': [], 'P46094': [], 'Q16570': [], 'O00590': [], 'P25106': [], 'Q9NPB9': [], 'O00421': [], 'P23945': [], 'P22888': [['643', 'S-palmitoyl cysteine'], ['644', 'S-palmitoyl cysteine']], 'P16473': [], 'Q8TCW9': [], 'Q8NFJ6': [], 'O14842': [], 'O15552': [], 'O14843': [], 'Q5NUL3': [], 'O15529': [], 'Q15722': [], 'Q9NPC1': [], 'Q9Y271': [], 'Q9NS75': [], 'Q8TDS5': [], 'Q92633': [], 'Q9HBW0': [['311', 'S-palmitoyl cysteine']], 'Q9UBY5': [['309', 'S-palmitoyl cysteine']], 'Q99677': [], 'Q9H1C0': [], 'P43657': [['284', 'S-palmitoyl cysteine']], 'P21453': [['328', 'S-palmitoyl cysteine']], 'O95136': [['305', 'S-palmitoyl cysteine']], 'Q99500': [], 'O95977': [['323', 'S-palmitoyl cysteine']], 'Q9H228': [['323', 'S-palmitoyl cysteine']], 'P21554': [['415', 'S-palmitoyl cysteine']], 'P34972': [], 'Q14330': [], 'Q9Y2T6': [], 'Q8TDV5': [], 'P25105': [], 'Q13258': [], 'Q9Y5Y4': [], 'P34995': [], 'P43116': [], 'P43115': [], 'P35408': [], 'P43088': [], 'P43119': [['308', 'S-palmitoyl cysteine'], ['311', 'S-palmitoyl cysteine'], ['383', 'S-farnesyl cysteine']], 'P21731': [], 'P48039': [], 'P49286': [], 'P30542': [['309', 'S-palmitoyl cysteine']], 'P29274': [], 'P29275': [['311', 'S-palmitoyl cysteine']], 'P0DMS8': [['303', 'S-palmitoyl cysteine']], 'P47900': [], 'P41231': [], 'P51582': [], 'Q15077': [], 'Q96G91': [], 'Q9H244': [], 'Q9BPV8': [], 'Q15391': [], 'Q8TDU6': [], 'Q99527': [], 'Q9BXC0': [], 'Q8TDS4': [], 'P49019': [], 'Q96P68': [], 'Q9BXA5': [], 'P08100': [['322', 'S-palmitoyl cysteine'], ['323', 'S-palmitoyl cysteine']], 'Q9UHM6': [], 'Q9H1Y3': [['325', 'S-palmitoyl cysteine']], 'Q6U736': [['315', 'S-palmitoyl cysteine'], ['316', 'S-palmitoyl cysteine']], 'P03999': [], 'P04001': [], 'P04000': [], 'P46091': [], 'P46089': [['313', 'S-palmitoyl cysteine']], 'P46093': [], 'P46095': [['345', 'S-palmitoyl cysteine']], 'P47775': [['317', 'S-palmitoyl cysteine']], 'P49685': [], 'Q13304': [], 'Q15760': [], 'Q99678': [], 'Q99679': [], 'Q99680': [], 'O00155': [], 'Q8NDV2': [], 'Q9NS67': [], 'O00270': [], 'O75388': [], 'Q49SQ1': [], 'Q9UPC5': [], 'Q9HC97': [], 'O15354': [], 'O60883': [], 'O43194': [], 'Q9Y5Y3': [], 'Q13585': [], 'Q9Y2T5': [], 'Q9BZJ8': [], 'Q9BZJ7': [], 'Q9BZJ6': [], 'Q8IYL9': [], 'Q15743': [], 'O95800': [], 'Q96P69': [], 'Q96P67': [], 'Q9NYM4': [], 'Q9NQS5': [], 'P60893': [], 'Q9BY21': [], 'Q9GZN0': [], 'Q96P66': [], 'Q9UNW8': [], 'Q8IZ08': [], 'Q6DWJ6': [], 'Q7Z602': [], 'Q7Z601': [], 'Q96CH1': [], 'Q8TDV2': [], 'Q86SP6': [], 'Q8NGU9': [], 'Q8TDV0': [], 'Q8TDT2': [], 'Q6NV75': [], 'Q9UJ42': [], 'Q8N6U8': [], 'Q16538': [], 'O14626': [], 'Q9NS66': [], 'Q9BXC1': [], 'Q14439': [], 'O15218': [], 'P32249': [], 'Q9BXB1': [], 'O75473': [], 'Q9HBX8': [], 'P04201': [], 'P35410': [], 'Q8TDS7': [], 'Q86SM8': [], 'Q96AM1': [], 'Q86SM5': [], 'Q96LB2': [], 'Q96LB1': [], 'Q96LB0': [], 'Q96LA9': [], 'Q86VZ1': [], 'O00398': [], 'Q9P1P5': [], 'O14804': [], 'Q96RI8': [], 'Q969N4': [], 'Q96RI9': [], 'Q16602': [], 'Q13324': [], 'Q02643': [], 'P48546': [], 'P43220': [], 'O95838': [], 'P47871': [], 'P47872': [], 'Q03431': [], 'P49190': [], 'P41586': [], 'P32241': [], 'P41587': [], 'O94910': [], 'O95490': [], 'Q9HAR2': [], 'Q9HBW9': [], 'Q14246': [], 'Q9UHX3': [], 'Q9BY15': [], 'P48960': [], 'Q86SQ6': [], 'Q96PE1': [], 'Q8IWK6': [], 'Q9NYQ6': [], 'Q9HCU4': [], 'Q9NYQ7': [], 'Q6QNK2': [], 'Q7Z7M1': [], 'Q5T601': [], 'Q8IZF7': [], 'Q8IZF5': [], 'Q8IZF3': [], 'Q8IZF2': [], 'O14514': [], 'O60241': [], 'O60242': [], 'Q9Y653': [], 'Q8IZP9': [], 'Q86Y34': [], 'Q8IZF6': [], 'Q8IZF4': [], 'Q86SQ4': [], 'Q96K78': [], 'Q8WXG9': [], 'P41180': [], 'Q9UBS5': [], 'O75899': [], 'Q13255': [], 'Q14416': [], 'Q14832': [], 'Q14833': [], 'P41594': [], 'O15303': [], 'Q14831': [], 'O00222': [], 'Q7RTX1': [], 'Q8TE23': [], 'Q7RTX0': [], 'Q8NFN8': [], 'Q5T848': [], 'Q6PRD1': [], 'Q8NFJ5': [], 'Q9NZH0': [], 'Q9NQ84': [], 'Q9NZD1': [], 'Q5T6X5': [], 'Q9UP38': [], 'Q14332': [], 'Q9NPG1': [], 'Q9ULV1': [], 'Q13467': [], 'O60353': [], 'O75084': [], 'Q9H461': [], 'O00144': [], 'Q9ULW2': [], 'Q9NYW7': [], 'Q9NYW6': [], 'Q9NYW5': [], 'Q9NYW4': [], 'Q9NYW3': [], 'Q9NYW2': [], 'Q9NYW1': [], 'Q9NYW0': [], 'Q9NYV9': [], 'Q9NYV8': [], 'Q9NYV7': [], 'P59542': [], 'P59543': [], 'P59541': [], 'P59538': [], 'P59533': [], 'P59534': [], 'P59535': [], 'P59536': [], 'Q7RTR8': [], 'P59537': [], 'P59539': [], 'P59540': [], 'P59544': [], 'P59551': [], 'Q5VW38': [], 'Q96N19': [], 'Q8TCB6': [], 'Q86W33': [], 'P51810': [], 'Q5UAW9': []}


### Disulfide bond
Cysteine residues participating in disulfide bonds. Jana explanation: typically the DB are formed in the extracellular loop in order to fix them and hept the binding with the ligand. They are not 'chains' but two positions whose Cysteine form a DB.

In [7]:
# Build the dictionary keys as the entry names, values are lists [position, ptmtype]
dicPTMdisBond=dict_ptm(os.path.join(resultspath,'remaining3/myprot_list_struc_remaining_merged_ENSG.csv'),'disBond')

P08100
Reponse from uniprot: Disulfide bond
DISULFID 110..187;  /evidence="ECO:0000255|PROSITE-ProRule:PRU00521, ECO:0000269|PubMed:26200343, ECO:0000269|PubMed:28753425"

['110..187;  /evidence="ECO:0000255|PROSITE-ProRule:PRU00521, ECO:0000269|PubMed:26200343, ECO:0000269|PubMed:28753425"\n']
Q9UHM6
Reponse from uniprot: Disulfide bond
DISULFID 143..221;  /evidence="ECO:0000255|PROSITE-ProRule:PRU00521"

['143..221;  /evidence="ECO:0000255|PROSITE-ProRule:PRU00521"\n']
Q9H1Y3
Reponse from uniprot: Disulfide bond
DISULFID 114..188;  /evidence="ECO:0000255|PROSITE-ProRule:PRU00521"

['114..188;  /evidence="ECO:0000255|PROSITE-ProRule:PRU00521"\n']
Q6U736
Reponse from uniprot: Disulfide bond
DISULFID 106..183;  /evidence="ECO:0000255|PROSITE-ProRule:PRU00521"

['106..183;  /evidence="ECO:0000255|PROSITE-ProRule:PRU00521"\n']
P03999
Reponse from uniprot: Disulfide bond
DISULFID 107..184;  /evidence="ECO:0000255|PROSITE-ProRule:PRU00521"

['107..184;  /evidence="ECO:0000255|PROSITE-ProRul

In [31]:
dicPTMdisBond={'P18089': [['85..164', 'Disulfide Bond']], 'P21917': [['108..185', 'Disulfide Bond'], ['372..375', 'Disulfide Bond']], 'P35367': [['100..180', 'Disulfide Bond'], ['441..444', 'Disulfide Bond']], 'P48145': [['109..188', 'Disulfide Bond']], 'P48146': [['117..197', 'Disulfide Bond']], 'Q99463': [['109..196', 'Disulfide Bond']], 'Q9NSD7': [['155..247', 'Disulfide Bond']], 'P32745': [['116..191', 'Disulfide Bond']], 'Q15722': [], 'Q9NPC1': [], 'P0DMS8': [['83..166', 'Disulfide Bond']], 'P46091': [['110..187', 'Disulfide Bond']], 'Q99678': [], 'Q99680': [], 'Q96CH1': [], 'Q8TDV2': [], 'P35410': [], 'Q9P1P4': [['104..189', 'Disulfide Bond']], 'O00398': [['106..181', 'Disulfide Bond']], 'Q9P1P5': [['116..201', 'Disulfide Bond']], 'O14804': [['99..192', 'Disulfide Bond']], 'Q96RI8': [['105..190', 'Disulfide Bond']], 'Q969N4': [['104..189', 'Disulfide Bond']], 'Q96RI9': [['105..190', 'Disulfide Bond']]}
# old {'P08908': [['109..187', 'Disulfide Bond']], 'P28222': [['122..199', 'Disulfide Bond']], 'P28221': [['111..188', 'Disulfide Bond']], 'P28566': [['95..173', 'Disulfide Bond']], 'P30939': [['96..172', 'Disulfide Bond']], 'P28223': [['148..227', 'Disulfide Bond'], ['349..353', 'Disulfide Bond']], 'P41595': [['128..207', 'Disulfide Bond'], ['350..353', 'Disulfide Bond']], 'P28335': [['127..207', 'Disulfide Bond'], ['337..341', 'Disulfide Bond']], 'Q13639': [['93..184', 'Disulfide Bond']], 'P47898': [['120..192', 'Disulfide Bond']], 'P50406': [['99..180', 'Disulfide Bond']], 'P34969': [['155..231', 'Disulfide Bond']], 'P11229': [['98..178', 'Disulfide Bond']], 'P08172': [['96..176', 'Disulfide Bond'], ['413..416\n', 'Disulfide Bond']], 'P20309': [['141..221', 'Disulfide Bond'], ['517..520', 'Disulfide Bond']], 'P08173': [['105..185', 'Disulfide Bond']], 'P08912': [['103..183', 'Disulfide Bond']], 'P35348': [['99..176', 'Disulfide Bond']], 'P35368': [['118..195', 'Disulfide Bond']], 'P25100': [], 'P08913': [['121..203', 'Disulfide Bond']], 'P18089': [['85..164', 'Disulfide Bond']], 'P18825': [['124..202', 'Disulfide Bond']], 'P08588': [['131..216', 'Disulfide Bond'], ['209..215', 'Disulfide Bond']], 'P07550': [['106..191', 'Disulfide Bond'], ['184..190\n', 'Disulfide Bond']], 'P13945': [['110..196', 'Disulfide Bond'], ['189..195', 'Disulfide Bond']], 'P21728': [['96..186', 'Disulfide Bond']], 'P14416': [['107..182', 'Disulfide Bond'], ['399..401', 'Disulfide Bond']], 'P35462': [['103..181', 'Disulfide Bond'], ['355..358', 'Disulfide Bond']], 'P21917': [['108..185', 'Disulfide Bond'], ['372..375', 'Disulfide Bond']], 'P21918': [['113..217', 'Disulfide Bond']], 'P35367': [['100..180', 'Disulfide Bond'], ['441..444', 'Disulfide Bond']], 'P25021': [['91..174', 'Disulfide Bond']], 'Q9Y5N1': [], 'Q9H3N8': [['87..164', 'Disulfide Bond']], 'Q96RJ0': [['96..182', 'Disulfide Bond']], 'P30556': [['101..180', 'Disulfide Bond']], 'P50052': [['35..290', 'Disulfide Bond'], ['117..195', 'Disulfide Bond']], 'P35414': [], 'P28336': [['116..198', 'Disulfide Bond']], 'P30550': [['113..196', 'Disulfide Bond']], 'P32247': [['120..203', 'Disulfide Bond']], 'P46663': [['110..189', 'Disulfide Bond']], 'P30411': [['130..211', 'Disulfide Bond']], 'P32238': [['18..29', 'Disulfide Bond'], ['114..196', 'Disulfide Bond']], 'P32239': [['127..205', 'Disulfide Bond']], 'Q16581': [['95..172', 'Disulfide Bond']], 'P21730': [['109..188', 'Disulfide Bond']], 'Q9P296': [['107..186', 'Disulfide Bond']], 'P25101': [['158..239', 'Disulfide Bond']], 'P24530': [['174..255', 'Disulfide Bond']], 'P21462': [['98..176', 'Disulfide Bond']], 'P25090': [['98..176', 'Disulfide Bond']], 'P25089': [['98..176', 'Disulfide Bond']], 'P47211': [['108..187', 'Disulfide Bond']], 'O43603': [['98..175', 'Disulfide Bond']], 'O60755': [['95..172', 'Disulfide Bond']], 'Q92847': [['116..198', 'Disulfide Bond']], 'P30968': [['114..196', 'Disulfide Bond']], 'Q969F8': [['115..191', 'Disulfide Bond']], 'Q99705': [], 'Q969V1': [], 'Q01726': [], 'Q01718': [], 'P41968': [], 'P32245': [['84', 'Disulfide Bond']], 'P33032': [], 'O43193': [['111..235', 'Disulfide Bond']], 'Q9HB89': [['134..219', 'Disulfide Bond']], 'Q9GZQ4': [['119..204', 'Disulfide Bond']], 'Q9GZQ6': [['116..203', 'Disulfide Bond']], 'Q9Y5X5': [['220..308', 'Disulfide Bond']], 'Q6W5P4': [['121..197', 'Disulfide Bond']], 'P48145': [['109..188', 'Disulfide Bond']], 'P48146': [['117..197', 'Disulfide Bond']], 'P25929': [['113..198', 'Disulfide Bond']], 'P49146': [['123..203', 'Disulfide Bond']], 'P50391': [['114..201', 'Disulfide Bond']], 'Q15761': [['114..198', 'Disulfide Bond']], 'Q99463': [['109..196', 'Disulfide Bond']], 'P30989': [['141..224', 'Disulfide Bond']], 'O95665': [['108..194', 'Disulfide Bond']], 'P41143': [['121..198', 'Disulfide Bond']], 'P41145': [['131..210', 'Disulfide Bond']], 'P35372': [['142..219', 'Disulfide Bond']], 'P41146': [['123..200', 'Disulfide Bond']], 'O43613': [['119..202', 'Disulfide Bond']], 'O43614': [['127..210', 'Disulfide Bond']], 'Q96P65': [], 'P49683': [['134..211', 'Disulfide Bond']], 'P25116': [['175..254', 'Disulfide Bond']], 'P55085': [['148..226', 'Disulfide Bond']], 'O00254': [['166..245', 'Disulfide Bond']], 'Q96RI0': [['149..228', 'Disulfide Bond']], 'Q9HBX9': [['27..40', 'Disulfide Bond'], ['34..53', 'Disulfide Bond'], ['47..62', 'Disulfide Bond'], ['485..563\n', 'Disulfide Bond']], 'Q8WXD0': [['45..58', 'Disulfide Bond'], ['52..71', 'Disulfide Bond'], ['65..80', 'Disulfide Bond'], ['495..573', 'Disulfide Bond']], 'Q9NSD7': [['155..247', 'Disulfide Bond']], 'Q8TDU9': [['114..191', 'Disulfide Bond']], 'P30872': [['130..208', 'Disulfide Bond']], 'P30874': [['115..193', 'Disulfide Bond']], 'P32745': [['116..191', 'Disulfide Bond']], 'P31391': [['119..198', 'Disulfide Bond']], 'P35346': [['112..186', 'Disulfide Bond']], 'P25103': [['105..180', 'Disulfide Bond']], 'P21452': [['106..181', 'Disulfide Bond']], 'P29371': [['158..233', 'Disulfide Bond']], 'P34981': [['98..179', 'Disulfide Bond']], 'Q9UKP6': [['123..199', 'Disulfide Bond']], 'P37288': [['124..203', 'Disulfide Bond']], 'P47901': [['107..186', 'Disulfide Bond']], 'P30518': [], 'P30559': [['112..187', 'Disulfide Bond']], 'Q99788': [['112..189', 'Disulfide Bond']], 'P32246': [['106..183', 'Disulfide Bond']], 'P41597': [['113..190', 'Disulfide Bond']], 'P51677': [['106..183', 'Disulfide Bond']], 'P51679': [['110..187', 'Disulfide Bond']], 'P51681': [['20..269', 'Disulfide Bond'], ['101..178', 'Disulfide Bond']], 'P51684': [['118..197', 'Disulfide Bond']], 'P32248': [['129..210', 'Disulfide Bond']], 'P51685': [['106..183', 'Disulfide Bond']], 'P51686': [['38..289', 'Disulfide Bond'], ['119..198', 'Disulfide Bond']], 'P46092': [['113..191', 'Disulfide Bond']], 'P25024': [['110..187', 'Disulfide Bond']], 'P25025': [['119..196', 'Disulfide Bond']], 'P49682': [['124..203', 'Disulfide Bond']], 'P61073': [['28..274', 'Disulfide Bond'], ['109..186', 'Disulfide Bond']], 'P32302': [['122..202', 'Disulfide Bond']], 'O00574': [['102..180', 'Disulfide Bond']], 'P49238': [['102..175', 'Disulfide Bond']], 'P46094': [['102..175', 'Disulfide Bond']], 'Q16570': [['51..276', 'Disulfide Bond'], ['129..195', 'Disulfide Bond']], 'O00590': [['117..195', 'Disulfide Bond']], 'P25106': [['117..196', 'Disulfide Bond']], 'Q9NPB9': [['112..184', 'Disulfide Bond']], 'O00421': [['103..181', 'Disulfide Bond']], 'P23945': [['18..25', 'Disulfide Bond'], ['23..32', 'Disulfide Bond'], ['275..346', 'Disulfide Bond'], ['276..356', 'Disulfide Bond'], ['276..292', 'Disulfide Bond'], ['292..338', 'Disulfide Bond'], ['442..517', 'Disulfide Bond']], 'P22888': [['439..514', 'Disulfide Bond']], 'P16473': [['31..41', 'Disulfide Bond'], ['494..569', 'Disulfide Bond']], 'Q8TCW9': [['137..217', 'Disulfide Bond']], 'Q8NFJ6': [['128..208', 'Disulfide Bond']], 'O14842': [['79..170', 'Disulfide Bond']], 'O15552': [], 'O14843': [['88..169', 'Disulfide Bond']], 'Q5NUL3': [['111..194', 'Disulfide Bond']], 'O15529': [], 'Q15722': [], 'Q9NPC1': [], 'Q9Y271': [['96..173', 'Disulfide Bond']], 'Q9NS75': [['111..187', 'Disulfide Bond']], 'Q8TDS5': [['165..237', 'Disulfide Bond']], 'Q92633': [['24..190', 'Disulfide Bond'], ['188..195', 'Disulfide Bond'], ['284..287', 'Disulfide Bond']], 'Q9HBW0': [], 'Q9UBY5': [], 'Q99677': [['111..188', 'Disulfide Bond']], 'Q9H1C0': [['94..175', 'Disulfide Bond']], 'P43657': [['89..168', 'Disulfide Bond']], 'P21453': [['184..191', 'Disulfide Bond'], ['282..287', 'Disulfide Bond']], 'O95136': [], 'Q99500': [], 'O95977': [], 'Q9H228': [], 'P21554': [], 'P34972': [], 'Q14330': [['94..172', 'Disulfide Bond']], 'Q9Y2T6': [], 'Q8TDV5': [], 'P25105': [['90..173', 'Disulfide Bond']], 'Q13258': [['105..183', 'Disulfide Bond']], 'Q9Y5Y4': [['104..182', 'Disulfide Bond']], 'P34995': [['110..188', 'Disulfide Bond']], 'P43116': [['109..187', 'Disulfide Bond']], 'P43115': [], 'P35408': [['92..170', 'Disulfide Bond']], 'P43088': [['108..186', 'Disulfide Bond']], 'P43119': [['5..165', 'Disulfide Bond'], ['92..170', 'Disulfide Bond']], 'P21731': [['105..183', 'Disulfide Bond']], 'P48039': [['100..177', 'Disulfide Bond']], 'P49286': [['113..190', 'Disulfide Bond']], 'P30542': [['80..169', 'Disulfide Bond']], 'P29274': [['71..159', 'Disulfide Bond'], ['74..146', 'Disulfide Bond'], ['77..166', 'Disulfide Bond'], ['259..262', 'Disulfide Bond']], 'P29275': [['78..171', 'Disulfide Bond']], 'P0DMS8': [['83..166', 'Disulfide Bond']], 'P47900': [['42..296', 'Disulfide Bond'], ['124..202', 'Disulfide Bond']], 'P41231': [['106..183', 'Disulfide Bond']], 'P51582': [['108..185', 'Disulfide Bond']], 'Q15077': [['99..177', 'Disulfide Bond']], 'Q96G91': [['102..180', 'Disulfide Bond']], 'Q9H244': [['17..270', 'Disulfide Bond'], ['97..175\n', 'Disulfide Bond']], 'Q9BPV8': [['116..194', 'Disulfide Bond']], 'Q15391': [['94..172', 'Disulfide Bond']], 'Q8TDU6': [['85..155', 'Disulfide Bond']], 'Q99527': [['130..207', 'Disulfide Bond']], 'Q9BXC0': [['88..165', 'Disulfide Bond']], 'Q8TDS4': [['100..177', 'Disulfide Bond']], 'P49019': [['100..177', 'Disulfide Bond']], 'Q96P68': [['106..183', 'Disulfide Bond']], 'Q9BXA5': [['95..172', 'Disulfide Bond']], 'P08100': [['110..187', 'Disulfide Bond']], 'Q9UHM6': [['143..221', 'Disulfide Bond']], 'Q9H1Y3': [['114..188', 'Disulfide Bond']], 'Q6U736': [['106..183', 'Disulfide Bond']], 'P03999': [['107..184', 'Disulfide Bond']], 'P04001': [['126..203', 'Disulfide Bond']], 'P04000': [['126..203', 'Disulfide Bond']], 'P46091': [['110..187', 'Disulfide Bond']], 'P46089': [], 'P46093': [['90..168', 'Disulfide Bond']], 'P46095': [], 'P47775': [], 'P49685': [], 'Q13304': [['132..209', 'Disulfide Bond']], 'Q15760': [['138..210', 'Disulfide Bond']], 'Q99678': [], 'Q99679': [], 'Q99680': [], 'O00155': [], 'Q8NDV2': [['79..156', 'Disulfide Bond']], 'Q9NS67': [['95..171', 'Disulfide Bond']], 'O00270': [], 'O75388': [['114..191', 'Disulfide Bond']], 'Q49SQ1': [['101..179', 'Disulfide Bond']], 'Q9UPC5': [['127..204', 'Disulfide Bond']], 'Q9HC97': [['89..162', 'Disulfide Bond']], 'O15354': [['334..419', 'Disulfide Bond']], 'O60883': [['203..286', 'Disulfide Bond']], 'O43194': [['11..191', 'Disulfide Bond'], ['108..210', 'Disulfide Bond']], 'Q9Y5Y3': [], 'Q13585': [['101..178', 'Disulfide Bond']], 'Q9Y2T5': [['114..193', 'Disulfide Bond']], 'Q9BZJ8': [], 'Q9BZJ7': [], 'Q9BZJ6': [], 'Q8IYL9': [['87..170', 'Disulfide Bond']], 'Q15743': [['94..172', 'Disulfide Bond']], 'O95800': [], 'Q96P69': [['79..156', 'Disulfide Bond']], 'Q96P67': [], 'Q9NYM4': [['144..224', 'Disulfide Bond']], 'Q9NQS5': [], 'P60893': [['94..172', 'Disulfide Bond']], 'Q9BY21': [['114..192', 'Disulfide Bond']], 'Q9GZN0': [], 'Q96P66': [['104..182', 'Disulfide Bond']], 'Q9UNW8': [['115..186', 'Disulfide Bond']], 'Q8IZ08': [], 'Q6DWJ6': [], 'Q7Z602': [], 'Q7Z601': [], 'Q96CH1': [], 'Q8TDV2': [], 'Q86SP6': [['105..182', 'Disulfide Bond']], 'Q8NGU9': [], 'Q8TDV0': [['111..187', 'Disulfide Bond']], 'Q8TDT2': [['104..182', 'Disulfide Bond']], 'Q6NV75': [], 'Q9UJ42': [], 'Q8N6U8': [['100..178', 'Disulfide Bond']], 'Q16538': [], 'O14626': [], 'Q9NS66': [['96..174', 'Disulfide Bond']], 'Q9BXC1': [['91..168', 'Disulfide Bond']], 'Q14439': [], 'O15218': [['126..202', 'Disulfide Bond']], 'P32249': [['104..181', 'Disulfide Bond']], 'Q9BXB1': [['29..35', 'Disulfide Bond'], ['33..43', 'Disulfide Bond'], ['339..364', 'Disulfide Bond'], ['470..522', 'Disulfide Bond'], ['471..476', 'Disulfide Bond'], ['618..693', 'Disulfide Bond']], 'O75473': [['34..40', 'Disulfide Bond'], ['38..52', 'Disulfide Bond'], ['348..373', 'Disulfide Bond'], ['479..541', 'Disulfide Bond'], ['637..712', 'Disulfide Bond']], 'Q9HBX8': [['642..717', 'Disulfide Bond']], 'P04201': [], 'P35410': [], 'Q8TDS7': [], 'Q86SM8': [], 'Q96AM1': [], 'Q86SM5': [], 'Q96LB2': [], 'Q96LB1': [], 'Q96LB0': [], 'Q96LA9': [], 'Q86VZ1': [], 'O00398': [['106..181', 'Disulfide Bond']], 'Q9P1P5': [['116..201', 'Disulfide Bond']], 'O14804': [['99..192', 'Disulfide Bond']], 'Q96RI8': [['105..190', 'Disulfide Bond']], 'Q969N4': [['104..189', 'Disulfide Bond']], 'Q96RI9': [['105..190', 'Disulfide Bond']], 'Q16602': [['48..74', 'Disulfide Bond'], ['65..105', 'Disulfide Bond'], ['88..127\n', 'Disulfide Bond']], 'Q13324': [['14..50', 'Disulfide Bond'], ['40..83', 'Disulfide Bond'], ['64..98', 'Disulfide Bond'], ['184..254', 'Disulfide Bond']], 'Q02643': [['41..64', 'Disulfide Bond'], ['55..96', 'Disulfide Bond'], ['78..112', 'Disulfide Bond']], 'P48546': [['46..70', 'Disulfide Bond'], ['61..103', 'Disulfide Bond'], ['84..118', 'Disulfide Bond']], 'P43220': [['46..71', 'Disulfide Bond'], ['62..104', 'Disulfide Bond'], ['85..126', 'Disulfide Bond'], ['226..296', 'Disulfide Bond']], 'O95838': [['83..105', 'Disulfide Bond'], ['96..137', 'Disulfide Bond'], ['118..159', 'Disulfide Bond']], 'P47871': [['43..67', 'Disulfide Bond'], ['58..100', 'Disulfide Bond'], ['81..121', 'Disulfide Bond'], ['224..294', 'Disulfide Bond']], 'P47872': [], 'Q03431': [['48..117', 'Disulfide Bond'], ['108..148', 'Disulfide Bond'], ['131..170', 'Disulfide Bond']], 'P49190': [], 'P41586': [['34..63', 'Disulfide Bond'], ['54..118', 'Disulfide Bond'], ['77..134\n', 'Disulfide Bond']], 'P32241': [['50..72', 'Disulfide Bond'], ['63..105', 'Disulfide Bond'], ['86..122', 'Disulfide Bond'], ['215..285', 'Disulfide Bond']], 'P41587': [['38..61', 'Disulfide Bond'], ['52..93', 'Disulfide Bond'], ['75..109', 'Disulfide Bond'], ['202..271', 'Disulfide Bond']], 'O94910': [['41..71', 'Disulfide Bond'], ['50..128', 'Disulfide Bond'], ['83..115', 'Disulfide Bond'], ['96..102', 'Disulfide Bond'], ['140..322', 'Disulfide Bond'], ['480..515', 'Disulfide Bond'], ['503..532', 'Disulfide Bond'], ['802..833', 'Disulfide Bond'], ['821..835', 'Disulfide Bond']], 'O95490': [['136..318', 'Disulfide Bond']], 'Q9HAR2': [['36..66', 'Disulfide Bond'], ['45..123', 'Disulfide Bond'], ['78..110', 'Disulfide Bond'], ['91..97', 'Disulfide Bond'], ['135..317', 'Disulfide Bond']], 'Q9HBW9': [['22..33', 'Disulfide Bond'], ['27..42', 'Disulfide Bond'], ['44..56', 'Disulfide Bond'], ['62..75', 'Disulfide Bond'], ['69..84', 'Disulfide Bond'], ['86..107', 'Disulfide Bond']], 'Q14246': [['35..47', 'Disulfide Bond'], ['41..56', 'Disulfide Bond'], ['58..78', 'Disulfide Bond'], ['84..97', 'Disulfide Bond'], ['91..106', 'Disulfide Bond'], ['108..130', 'Disulfide Bond'], ['136..148', 'Disulfide Bond'], ['142..157', 'Disulfide Bond'], ['159..170', 'Disulfide Bond'], ['176..188', 'Disulfide Bond'], ['182..197', 'Disulfide Bond'], ['199..219', 'Disulfide Bond'], ['225..235', 'Disulfide Bond'], ['229..244', 'Disulfide Bond'], ['246..266', 'Disulfide Bond'], ['272..285', 'Disulfide Bond'], ['279..294', 'Disulfide Bond'], ['296..315', 'Disulfide Bond']], 'Q9UHX3': [['29..39', 'Disulfide Bond'], ['33..45', 'Disulfide Bond'], ['47..65', 'Disulfide Bond'], ['71..85', 'Disulfide Bond'], ['79..94', 'Disulfide Bond'], ['96..117', 'Disulfide Bond'], ['123..136', 'Disulfide Bond'], ['130..145', 'Disulfide Bond'], ['147..161', 'Disulfide Bond'], ['167..180', 'Disulfide Bond'], ['174..189', 'Disulfide Bond'], ['191..210', 'Disulfide Bond'], ['216..229', 'Disulfide Bond'], ['223..238', 'Disulfide Bond'], ['240..259', 'Disulfide Bond']], 'Q9BY15': [['28..37', 'Disulfide Bond'], ['31..43', 'Disulfide Bond'], ['45..65', 'Disulfide Bond'], ['71..85', 'Disulfide Bond'], ['79..94', 'Disulfide Bond'], ['96..117', 'Disulfide Bond']], 'P48960': [['26..36', 'Disulfide Bond'], ['30..42', 'Disulfide Bond'], ['44..62', 'Disulfide Bond'], ['68..82', 'Disulfide Bond'], ['76..91', 'Disulfide Bond'], ['93..114', 'Disulfide Bond'], ['120..133', 'Disulfide Bond'], ['127..142', 'Disulfide Bond'], ['144..158', 'Disulfide Bond'], ['164..177', 'Disulfide Bond'], ['171..186', 'Disulfide Bond'], ['188..207', 'Disulfide Bond'], ['213..226', 'Disulfide Bond'], ['220..235', 'Disulfide Bond'], ['237..256', 'Disulfide Bond']], 'Q86SQ6': [], 'Q96PE1': [['268..328', 'Disulfide Bond']], 'Q8IWK6': [['264..324', 'Disulfide Bond']], 'Q9NYQ6': [['1307..1318', 'Disulfide Bond'], ['1312..1349', 'Disulfide Bond'], ['1351..1360', 'Disulfide Bond'], ['1367..1378', 'Disulfide Bond'], ['1372..1387', 'Disulfide Bond'], ['1389..1398', 'Disulfide Bond'], ['1407..1418', 'Disulfide Bond'], ['1412..1428', 'Disulfide Bond'], ['1430..1440', 'Disulfide Bond'], ['1620..1646', 'Disulfide Bond'], ['1653..1664', 'Disulfide Bond'], ['1658..1673', 'Disulfide Bond'], ['1675..1684', 'Disulfide Bond'], ['1840..1870', 'Disulfide Bond'], ['1876..1887', 'Disulfide Bond'], ['1881..1896', 'Disulfide Bond'], ['1898..1907', 'Disulfide Bond'], ['1911..1922', 'Disulfide Bond'], ['1916..1934', 'Disulfide Bond'], ['1936..1945', 'Disulfide Bond'], ['1953..1966', 'Disulfide Bond'], ['1968..1978', 'Disulfide Bond'], ['1985..2000', 'Disulfide Bond'], ['1987..2003', 'Disulfide Bond'], ['2005..2015', 'Disulfide Bond'], ['2024..2033', 'Disulfide Bond'], ['2036..2048', 'Disulfide Bond']], 'Q9HCU4': [['1232..1243', 'Disulfide Bond'], ['1237..1274', 'Disulfide Bond'], ['1276..1285', 'Disulfide Bond'], ['1292..1303', 'Disulfide Bond'], ['1297..1312', 'Disulfide Bond'], ['1314..1323', 'Disulfide Bond'], ['1332..1343', 'Disulfide Bond'], ['1337..1353', 'Disulfide Bond'], ['1355..1365', 'Disulfide Bond'], ['1545..1571', 'Disulfide Bond'], ['1578..1589', 'Disulfide Bond'], ['1583..1598', 'Disulfide Bond'], ['1600..1609', 'Disulfide Bond'], ['1761..1791', 'Disulfide Bond'], ['1797..1808', 'Disulfide Bond'], ['1802..1817', 'Disulfide Bond'], ['1819..1828', 'Disulfide Bond'], ['1832..1843', 'Disulfide Bond'], ['1837..1855', 'Disulfide Bond'], ['1857..1866', 'Disulfide Bond'], ['1887..1899', 'Disulfide Bond'], ['1889..1906', 'Disulfide Bond'], ['1908..1921', 'Disulfide Bond'], ['1924..1936', 'Disulfide Bond'], ['1926..1943', 'Disulfide Bond'], ['1945..1954', 'Disulfide Bond'], ['1957..1969', 'Disulfide Bond']], 'Q9NYQ7': [['1379..1390', 'Disulfide Bond'], ['1384..1421', 'Disulfide Bond'], ['1423..1432', 'Disulfide Bond'], ['1439..1450', 'Disulfide Bond'], ['1444..1459', 'Disulfide Bond'], ['1461..1470', 'Disulfide Bond'], ['1479..1490', 'Disulfide Bond'], ['1484..1500', 'Disulfide Bond'], ['1502..1513', 'Disulfide Bond'], ['1693..1719', 'Disulfide Bond'], ['1726..1737', 'Disulfide Bond'], ['1731..1746', 'Disulfide Bond'], ['1748..1757', 'Disulfide Bond'], ['1915..1944', 'Disulfide Bond'], ['1950..1961', 'Disulfide Bond'], ['1955..1970', 'Disulfide Bond'], ['1972..1981', 'Disulfide Bond'], ['1985..1996', 'Disulfide Bond'], ['1990..2008', 'Disulfide Bond'], ['2010..2019', 'Disulfide Bond'], ['2027..2040', 'Disulfide Bond'], ['2042..2052', 'Disulfide Bond'], ['2059..2074', 'Disulfide Bond'], ['2061..2077', 'Disulfide Bond'], ['2079..2089', 'Disulfide Bond'], ['2098..2107', 'Disulfide Bond'], ['2110..2122', 'Disulfide Bond']], 'Q6QNK2': [], 'Q7Z7M1': [['146..212', 'Disulfide Bond']], 'Q5T601': [], 'Q8IZF7': [], 'Q8IZF5': [], 'Q8IZF3': [], 'Q8IZF2': [['293..350', 'Disulfide Bond'], ['391..449', 'Disulfide Bond'], ['492..545', 'Disulfide Bond']], 'O14514': [['273..309', 'Disulfide Bond'], ['277..314', 'Disulfide Bond'], ['288..299', 'Disulfide Bond'], ['366..400', 'Disulfide Bond'], ['370..406', 'Disulfide Bond'], ['381..390', 'Disulfide Bond'], ['421..456', 'Disulfide Bond'], ['425..461', 'Disulfide Bond'], ['436..446', 'Disulfide Bond'], ['479..514', 'Disulfide Bond'], ['483..519', 'Disulfide Bond'], ['494..504', 'Disulfide Bond'], ['534..569', 'Disulfide Bond'], ['538..574', 'Disulfide Bond'], ['549..559', 'Disulfide Bond'], ['581..616', 'Disulfide Bond'], ['604..634', 'Disulfide Bond'], ['884..921', 'Disulfide Bond'], ['909..923', 'Disulfide Bond']], 'O60241': [['321..355', 'Disulfide Bond'], ['325..361', 'Disulfide Bond'], ['336..345', 'Disulfide Bond'], ['376..411', 'Disulfide Bond'], ['380..416', 'Disulfide Bond'], ['391..401', 'Disulfide Bond'], ['431..466', 'Disulfide Bond'], ['435..471', 'Disulfide Bond'], ['446..456', 'Disulfide Bond'], ['487..522', 'Disulfide Bond'], ['491..527', 'Disulfide Bond'], ['502..512', 'Disulfide Bond'], ['534..569', 'Disulfide Bond'], ['557..587', 'Disulfide Bond'], ['874..906', 'Disulfide Bond'], ['894..908', 'Disulfide Bond']], 'O60242': [['30..55', 'Disulfide Bond'], ['303..336', 'Disulfide Bond'], ['307..342', 'Disulfide Bond'], ['318..326', 'Disulfide Bond'], ['357..392', 'Disulfide Bond'], ['361..397', 'Disulfide Bond'], ['372..382', 'Disulfide Bond'], ['412..447', 'Disulfide Bond'], ['416..452', 'Disulfide Bond'], ['427..437', 'Disulfide Bond'], ['467..502', 'Disulfide Bond'], ['471..507', 'Disulfide Bond'], ['482..492', 'Disulfide Bond'], ['514..549', 'Disulfide Bond'], ['537..567', 'Disulfide Bond'], ['819..851', 'Disulfide Bond'], ['839..853', 'Disulfide Bond']], 'Q9Y653': [], 'Q8IZP9': [], 'Q86Y34': [], 'Q8IZF6': [['58..123', 'Disulfide Bond']], 'Q8IZF4': [], 'Q86SQ4': [['41..67', 'Disulfide Bond'], ['94..111', 'Disulfide Bond'], ['186..254', 'Disulfide Bond']], 'Q96K78': [], 'Q8WXG9': [], 'P41180': [['60..101', 'Disulfide Bond'], ['129', 'Disulfide Bond'], ['131', 'Disulfide Bond'], ['236..561', 'Disulfide Bond'], ['358..395', 'Disulfide Bond'], ['437..449', 'Disulfide Bond'], ['542..562', 'Disulfide Bond'], ['546..565', 'Disulfide Bond'], ['568..582', 'Disulfide Bond'], ['585..598', 'Disulfide Bond']], 'Q9UBS5': [['100..145', 'Disulfide Bond'], ['131..157', 'Disulfide Bond'], ['220..246', 'Disulfide Bond'], ['376..410', 'Disulfide Bond']], 'O75899': [['108..135', 'Disulfide Bond'], ['237..266', 'Disulfide Bond'], ['265..302\n', 'Disulfide Bond']], 'Q13255': [['67..109', 'Disulfide Bond'], ['140', 'Disulfide Bond'], ['289..291', 'Disulfide Bond'], ['378..394', 'Disulfide Bond'], ['432..439', 'Disulfide Bond'], ['657..746\n', 'Disulfide Bond']], 'Q14416': [['50..92', 'Disulfide Bond'], ['234..518', 'Disulfide Bond'], ['355..362', 'Disulfide Bond'], ['400..407', 'Disulfide Bond'], ['500..519', 'Disulfide Bond'], ['504..522', 'Disulfide Bond'], ['525..537', 'Disulfide Bond'], ['540..553', 'Disulfide Bond']], 'Q14832': [['57..99', 'Disulfide Bond'], ['240..527', 'Disulfide Bond'], ['361..373', 'Disulfide Bond'], ['412..419', 'Disulfide Bond'], ['509..528', 'Disulfide Bond'], ['513..531', 'Disulfide Bond'], ['534..546', 'Disulfide Bond'], ['549..562', 'Disulfide Bond']], 'Q14833': [['67..109', 'Disulfide Bond'], ['249..538', 'Disulfide Bond'], ['372..388', 'Disulfide Bond'], ['428..435', 'Disulfide Bond'], ['520..539', 'Disulfide Bond'], ['524..542', 'Disulfide Bond'], ['545..557', 'Disulfide Bond'], ['560..573', 'Disulfide Bond']], 'P41594': [['57..99', 'Disulfide Bond'], ['241..530', 'Disulfide Bond'], ['276..278', 'Disulfide Bond'], ['365..381', 'Disulfide Bond'], ['419..426', 'Disulfide Bond'], ['511..531', 'Disulfide Bond'], ['515..534', 'Disulfide Bond'], ['537..549', 'Disulfide Bond'], ['552..565', 'Disulfide Bond'], ['644..733\n', 'Disulfide Bond']], 'O15303': [['57..99', 'Disulfide Bond'], ['244..536', 'Disulfide Bond'], ['367..383', 'Disulfide Bond'], ['423..430', 'Disulfide Bond'], ['518..537', 'Disulfide Bond'], ['522..540', 'Disulfide Bond'], ['543..555', 'Disulfide Bond'], ['558..571', 'Disulfide Bond']], 'Q14831': [['67..109', 'Disulfide Bond'], ['249..541', 'Disulfide Bond'], ['374..390', 'Disulfide Bond'], ['430..437', 'Disulfide Bond'], ['523..542', 'Disulfide Bond'], ['527..545', 'Disulfide Bond'], ['548..560', 'Disulfide Bond'], ['563..576', 'Disulfide Bond']], 'O00222': [['64..106', 'Disulfide Bond'], ['246..534', 'Disulfide Bond'], ['369..384', 'Disulfide Bond'], ['424..431', 'Disulfide Bond'], ['516..535', 'Disulfide Bond'], ['520..538', 'Disulfide Bond'], ['541..553', 'Disulfide Bond'], ['556..569', 'Disulfide Bond']], 'Q7RTX1': [], 'Q8TE23': [], 'Q7RTX0': [], 'Q8NFN8': [], 'Q5T848': [], 'Q6PRD1': [], 'Q8NFJ5': [], 'Q9NZH0': [], 'Q9NQ84': [], 'Q9NZD1': [], 'Q5T6X5': [['131', 'Disulfide Bond']], 'Q9UP38': [['116..177', 'Disulfide Bond'], ['124..170', 'Disulfide Bond'], ['161..198', 'Disulfide Bond'], ['187..227', 'Disulfide Bond'], ['191..215', 'Disulfide Bond']], 'Q14332': [['39..100', 'Disulfide Bond'], ['47..93', 'Disulfide Bond'], ['84..121', 'Disulfide Bond'], ['110..150', 'Disulfide Bond'], ['114..138', 'Disulfide Bond']], 'Q9NPG1': [['28..89', 'Disulfide Bond'], ['36..82', 'Disulfide Bond'], ['73..110', 'Disulfide Bond'], ['99..133', 'Disulfide Bond'], ['103..127', 'Disulfide Bond']], 'Q9ULV1': [['45..106', 'Disulfide Bond'], ['53..99', 'Disulfide Bond'], ['90..128', 'Disulfide Bond'], ['117..158', 'Disulfide Bond'], ['121..145', 'Disulfide Bond'], ['181..200', 'Disulfide Bond'], ['204..282', 'Disulfide Bond'], ['302..377', 'Disulfide Bond']], 'Q13467': [['33..94', 'Disulfide Bond'], ['41..87', 'Disulfide Bond'], ['78..116', 'Disulfide Bond'], ['105..147', 'Disulfide Bond'], ['109..133', 'Disulfide Bond']], 'O60353': [['24..85', 'Disulfide Bond'], ['32..78', 'Disulfide Bond'], ['69..106', 'Disulfide Bond'], ['95..129', 'Disulfide Bond'], ['99..123', 'Disulfide Bond']], 'O75084': [['49..110', 'Disulfide Bond'], ['57..103', 'Disulfide Bond'], ['94..131', 'Disulfide Bond'], ['120..160', 'Disulfide Bond'], ['124..148', 'Disulfide Bond']], 'Q9H461': [['35..96', 'Disulfide Bond'], ['43..89', 'Disulfide Bond'], ['80..118', 'Disulfide Bond'], ['107..148', 'Disulfide Bond'], ['111..135', 'Disulfide Bond']], 'O00144': [['39..100', 'Disulfide Bond'], ['47..93', 'Disulfide Bond'], ['84..122', 'Disulfide Bond'], ['111..152', 'Disulfide Bond'], ['115..139', 'Disulfide Bond']], 'Q9ULW2': [['34..95', 'Disulfide Bond'], ['42..88', 'Disulfide Bond'], ['79..117', 'Disulfide Bond'], ['106..147', 'Disulfide Bond'], ['110..134', 'Disulfide Bond']], 'Q9NYW7': [], 'Q9NYW6': [], 'Q9NYW5': [], 'Q9NYW4': [], 'Q9NYW3': [], 'Q9NYW2': [], 'Q9NYW1': [], 'Q9NYW0': [], 'Q9NYV9': [], 'Q9NYV8': [], 'Q9NYV7': [], 'P59542': [], 'P59543': [], 'P59541': [], 'P59538': [], 'P59533': [], 'P59534': [], 'P59535': [], 'P59536': [], 'Q7RTR8': [], 'P59537': [], 'P59539': [], 'P59540': [], 'P59544': [], 'P59551': [], 'Q5VW38': [['109..228', 'Disulfide Bond']], 'Q96N19': [], 'Q8TCB6': [['99..191', 'Disulfide Bond']], 'Q86W33': [], 'P51810': [], 'Q5UAW9': []}
# Q13324-2:51 erased

## Save data into file
Once each dictionary has been created, we build a file into the folder 'PTM_IMPACT' with the entry name, position, PTM and PTM Type.

In [8]:
# first build a file without the DB information
def is_ptm(dictionary,protName,pos):
    '''Return the type of PTM in case the position is in the dictionary of PTMs'''
    ptmList=dictionary[protName]
    posType={}# build dict with the list of list 
    for element in ptmList:# list of lists [pos, ptmType]
        posType[element[0]]=element[1] # position: type
#     print(posType)
    if str(pos) in posType.keys():# if the position is some PTM
        return posType[str(pos)]# return the PTM type
    else:
        return None
# REVISAR SI DA EL MISMO ERROR QUE EN EL D
# is_ptm(dicPTMmodRes,'P08908','11')  
    
def check_ptm(protName,pos):
    '''Build a list to be the 'next' line of the PTM file '''
    ptmNewList=[]
    ptmNewList.append(protName)# first element the Uniprot code
    ptmNewList.append(pos) # second the seqNum
    
    dicts=[dicPTMmodRes,dicPTMglyco,dicPTMlipid]
    # check the position in every dictionary:
    ptmNewList.append('')#ptm, Yes/NO, will be fullfilled later 
    for dictionary in dicts: 
        ptm=is_ptm(dictionary,protName,pos) # [PTM type, ECO] or None        
        if ptm: # the position correspond to a PTM of the dictionary
            ptmNewList.append(ptm) # write the description into the corresponding column
        else:
            ptmNewList.append('')
    # yes or no in PTM columns        
    if ptmNewList[3]!='' or ptmNewList[4]!='' or ptmNewList[5]!='':
        ptmNewList[2]='Yes'
    else:
        ptmNewList[2]='No'
    return ptmNewList
    
# print(check_ptm('Q5NUL3',360,dicPTM))
        
def store_ptm(old,new):
    '''Build a new file with the Uniprot IDs, the SeqNums and the possible PTM types'''
    with open(old,"r") as myvarfile:
        myvarread = csv.reader(myvarfile, delimiter=';') 
        with open(new,"w") as myvarfile_pred: 
            mywriter = csv.writer(myvarfile_pred, delimiter=';') 
            for myrow in myvarread:
                if myrow[1]=="Family": #first row, new names
                    row=['Entry','Pos','PTM','Modified Residue','Glycosylation','Lipidation']
                    mywriter.writerow(row)
                    continue            
                else:
                    newrow=['','','','','','']
                    uniprot=myrow[4]
                    seqNum=myrow[10]
#                     print(uniprot,seqNum)# Uniprot ID, SequenceNumber from the 'big' file
                    # copy the Uniprot ID, SequenceNumber into the ptm file

                    newrow=check_ptm(uniprot,seqNum)#entry, position
#                     print(newrow)
                mywriter.writerow(newrow)
    return print('File OK')
                        
variants_aff_pos=os.path.join(resultspath,"remaining3/PTM_IMPACT/vars_A_remaining_impact_role1.csv")
variants_aff_pos_pred=os.path.join(resultspath,"remaining3/PTM_IMPACT/PTMs_remaining_without_disBond.csv")
store_ptm(variants_aff_pos,variants_aff_pos_pred) 

File OK


In [9]:
#db work
#Now add the disulfide bond information into the file 
def rearrange_dbDict(dictDB):
    '''new dictonary protein: [[pos1,ECO],[pos2,ECO],[pos3,ECO],...] '''
    newDictDB={}
    for protein in dictDB: #iterate over each dictionary
        newDictDB[protein]=[]
        listOfSeqs=dictDB[protein]# list os lists of sequences
        startsEnds=[] #pos1, pos2, eco
        for element in listOfSeqs:

            seqs=element[0]# sequence of each DB of the protein
            stEnd=seqs.split('..')# [start position, end position]
            if len(stEnd)>1 and stEnd[1].count('\n')>0:
                aux=stEnd[1]
                aux1=aux.replace('\n','')
                stEnd[1]=aux1
            for position in stEnd:
#                 print(position)
                startsEnds.append(int(position))
#             print(eco)

            newDictDB[protein].append(startsEnds)# uniprot: [[st1,end1, eco1],[st2,end2, eco2],[st3,end3,eco3],...]        
            startsEnds=[]
    return newDictDB

dicPTMDB=rearrange_dbDict(dicPTMdisBond)
print(dicPTMDB)

{'P08100': [[110, 187]], 'Q9UHM6': [[143, 221]], 'Q9H1Y3': [[114, 188]], 'Q6U736': [[106, 183]], 'P03999': [[107, 184]], 'P04001': [[126, 203]], 'P04000': [[126, 203]], 'Q9P1Q5': [[97, 189]], 'P47890': [[97, 189]], 'Q8NH01': [[94, 186]]}


In [10]:
#Now add the disulfide bond information into the file 
def rearrange_dbDict(dictDB):
    '''new dictonary protein: [pos1,pos2,pos3,...] '''
    newDictDB={}
    for protein in dictDB: #iterate over each dictionary
        newDictDB[protein]=[]
        listOfSeqs=dictDB[protein]# list os lists of sequences

        for element in listOfSeqs:
            seqs=element[0]# sequence of each DB of the protein
            stEnd=seqs.split('..')# [start position, end position]
            if len(stEnd)>1 and stEnd[1].count('\n')>0:# the 2nd number can have shit that will be removed
                aux=stEnd[1]
                aux1=aux.replace('\n','')
                stEnd[1]=aux1

#             print(stEnd)
            for item in stEnd:
                newDictDB[protein].append(item)# uniprot: pos1,pos2,pos3,...]        
            startsEnds=[]
    return newDictDB

dicPTMDB=rearrange_dbDict(dicPTMdisBond)
print(dicPTMDB)

{'P08100': ['110', '187'], 'Q9UHM6': ['143', '221'], 'Q9H1Y3': ['114', '188'], 'Q6U736': ['106', '183'], 'P03999': ['107', '184'], 'P04001': ['126', '203'], 'P04000': ['126', '203'], 'Q9P1Q5': ['97', '189'], 'P47890': ['97', '189'], 'Q8NH01': ['94', '186']}



 REVISAR DESDE AQUI

In [11]:
def get_disBond(listSeq, pos):
    '''As an input we have the [pos1,pos2,pos3,...] for the
    protein and the position we are testing.
    Return '' or 'DB' value if the position is part of a DB'''
    # transfor elements into integers
    int_list = [int(i) for i in listSeq]
    if pos in int_list:
#         print('Match', pos, int_list)
        return 'DB'
    else:
        return None
    
def store_ptm_all(old,new):
    '''Add DB if the position is in a disulfide bond and the ECO next to it, change 'no' to 
    'yes' of the PTM column'''
    with open(old,"r") as myvarfile:
        myvarread = csv.reader(myvarfile, delimiter=';') 
        with open(new,"w") as myvarfile_pred: 
            mywriter = csv.writer(myvarfile_pred, delimiter=';') 
            for myrow in myvarread:
                if myrow[0]=="Entry": #first row, new names
                    row=myrow
                    row.append('Disulfide Bond')
                    mywriter.writerow(row)
                    continue            
                else:
                    uniprot=myrow[0]#uniprot
                    seqNum=myrow[1]#seqNum
                    newrow=myrow
#                     print(uniprot)
                    db=get_disBond(dicPTMDB[uniprot],int(seqNum))#'' or 'DB'
                    if db:
                        newrow.append(db)
                        newrow[2]='yes'

                mywriter.writerow(newrow)
                    
                    
    return print('The file',variants_aff_pos_pred,'has been successfully created.')

variants_aff_pos=os.path.join(resultspath,"remaining3/PTM_IMPACT/PTMs_remaining_without_disBond.csv")
variants_aff_pos_pred=os.path.join(resultspath,"remaining3/PTM_IMPACT/PTMs_remaining_all.csv")
store_ptm_all(variants_aff_pos,variants_aff_pos_pred)

The file /home/martalo/Documentos/TFM/GPCR_variants/Results/studied_GPCR_vars/remaining3/PTM_IMPACT/PTMs_remaining_all.csv has been successfully created.


## Save data into the 'big' file of variants
Insert the data fro PTMs_all.csv into the vars_A_impact_ptm_role2.csv file.

Al final he copiado manualmente las columnas de PTM_all.csv a vars_A_impact_ptm_role2.csv.